In [1]:
import meshplot as mp
import numpy as np

In [2]:
def hexs2tets(hexs, diagmode=0):
    tets = np.empty((len(hexs)*5,4),dtype=np.int)
    T2H = np.empty((len(hexs)*5,),dtype=np.int)
    for i in range(len(hexs)):
        if diagmode == 0:
            diagtype = np.random.randint(2)
        elif diagmode == 1:
            diagtype = 0
        else:
            diagtype = i % 2 == 0
        
        if diagtype == 0:
            tets[5*i + 0] = [hexs[i][j] for j in [0,1,2,5]]
            tets[5*i + 1] = [hexs[i][j] for j in [0,2,3,7]]
            tets[5*i + 2] = [hexs[i][j] for j in [2,6,7,5]]
            tets[5*i + 3] = [hexs[i][j] for j in [0,5,7,4]]
            tets[5*i + 4] = [hexs[i][j] for j in [0,2,7,5]]
        else:
            tets[5*i + 0] = [hexs[i][j] for j in [0,1,3,4]]
            tets[5*i + 1] = [hexs[i][j] for j in [1,2,3,6]]
            tets[5*i + 2] = [hexs[i][j] for j in [3,6,7,4]]
            tets[5*i + 3] = [hexs[i][j] for j in [1,6,4,5]]
            tets[5*i + 4] = [hexs[i][j] for j in [1,6,3,4]]
        T2H[5*i:5*(i+1)] = i
    return tets, T2H
        

def tets2tris(tets):
    tris = np.empty((len(tets)*4,3),dtype=np.int)
    for i in range(len(tets)):
        tris[4*i + 0] = [tets[i][j] for j in [0,1,2]]
        tris[4*i + 1] = [tets[i][j] for j in [0,1,3]]
        tris[4*i + 2] = [tets[i][j] for j in [1,2,3]]
        tris[4*i + 3] = [tets[i][j] for j in [2,0,3]]
    return tris

def generate_hex_grid(DX,N,center=True):
    n,m,l = N
    di,dj,dk = DX
    V = np.empty(((n+1)*(m+1)*(l+1), 3), dtype=np.float)
    
#     X,Y,Z = np.meshgrid(np.linspace(0,di*n,n+1),np.linspace(0,dj*m,m+1),np.linspace(0,dk*l,l+1))
#     V[:,0] = X.reshape(V.shape[0])
#     V[:,1] = Y.reshape(V.shape[0])
#     V[:,2] = Z.reshape(V.shape[0])
        
    for i in range(n+1):
        I = i*(m+1)*(l+1)
        for j in range(m+1):
            J = j*(l+1)
            for k in range(l+1):
                V[I+J+k] = [i*di,j*dj,k*dk]
        
    if center:
        V[:] -= [di*n*0.5,dj*m*0.5,dk*l*0.5]
        
    def vix(i,j,k):
        return i*(m+1)*(l+1) + j*(l+1) + k
        
    H = np.empty((n*m*l,8),dtype=np.int)
    for i in range(n):
        I = i*m*l
        for j in range(m):
            J = j*l
            for k in range(l):
                H[I+J+k] = [vix(i+0,j+0,k+0),
                            vix(i+1,j+0,k+0),
                            vix(i+1,j+0,k+1),
                            vix(i+0,j+0,k+1),
                            vix(i+0,j+1,k+0),
                            vix(i+1,j+1,k+0),
                            vix(i+1,j+1,k+1),
                            vix(i+0,j+1,k+1)]
                
    # generate periodic vertex pairs [ixA, ixB]
    # for each z-slice 
    #  BL -> TL | TR | BR    # == 3
    #  B !L!R -> T !L!R      # == (n-1)
    #  L !B!T -> R !B!T      # == (m-1)
    P = np.empty(((l+1)*(3+(n-1)+(m-1)),2),dtype=np.int)
    ix = 0
    for k in range(l+1):
        P[ix + 0] = [vix(0,0,k), vix(0,m,k)] # BL -> TL
        P[ix + 1] = [vix(0,0,k), vix(n,m,k)] # BL -> TR
        P[ix + 2] = [vix(0,0,k), vix(n,0,k)] # BL -> BR
        ix += 3
        # B !L!R -> T !L!R
        for i in range(1,n):
            P[ix] = [vix(i,0,k), vix(i,m,k)]
            ix += 1
        #  L !B!T -> R !B!T
        for j in range(1,m):
            P[ix] = [vix(0,j,k), vix(n,j,k)]
            ix += 1
    assert(ix == (l+1)*(3+(n-1)+(m-1)))
    
    return V, H, P

# note default set existing: allElements
def vega_file_string(V,T,M,S,P):
    assert(len(S)==len(M))
    nverts = len(V)
    ntets = len(T)
    setname = lambda i: "set_" + str(i)
    matname = lambda i: "mat_" + str(i)
    
    # # Vega mesh file.
    # # NUMVERT vertices, NUMTET elements
    s = "# Vega mesh file\n# %d vertices, %d elements\n" % (nverts, ntets)
    s += "\n"
    
    # *VERTICES
    # NUMVERTS 3 0 0
    # [VIX X Y Z # VIX STARTING AT 0(or 1)]_rows
    s += "*VERTICES\n%d 3 0 0\n" % nverts
    for i,v in enumerate(V):
        s += "%d %f %f %f\n" % (i, v[0], v[1], v[2])
    s += "\n"
    
    # *ELEMENTS
    # TET
    # NUMTETS 4 0
    # [TIX VIX0 VIX1 VIX2 VIX3]_rows
    s += "*ELEMENTS\nTET\n%d 4 0\n" % ntets
    for i,t in enumerate(T):
        s += "%d %d %d %d %d\n" % (i, t[0], t[1], t[2], t[3])
    s += "\n"
        
    # # MATERIALNAMEi material
    # *MATERIAL MATERIALNAMEi
    # ENU, 1000, 25000000, 0.45 # density (kg/m^3), Young's modulus (N/m^2), Poisson's ratio
    for i,m in enumerate(M):
        s += "*MATERIAL %s\n" % matname(i)
        s += "ENU, %g, %g, %g\n\n" % (m[0],m[1],m[2])

    # # SETNAMEi elements
    # *SET SETNAMEi
    # TIX0, TIX1, TIX2 # sorted commaseparated indices (linebreaks ignored)
    # TIX3, TIX4, ...
    for i,st in enumerate(S):
        name = setname(i)
        s += "# %s elements\n*SET %s\n" % (name, name)
        rows = []
        for row in range(len(st)//8):
            rows.append(", ".join([str(st[row*8+j]) for j in range(8)]))
        nrest = len(st) % 8
        if nrest > 0:
            rows.append(", ".join([str(st[len(st)-nrest+j]) for j in range(nrest)]))
        s += ",\n".join(rows)
        s += "\n\n"

    # # link SETNAMEi to MATERIALNAMEi
    # *REGION
    # SETNAMEi, MATERIALNAMEi
    for i in range(len(M)):
        s += "# link %s to %s\n" % (setname(i), matname(i))
        s += "*REGION\n%s, %s\n\n" % (setname(i), matname(i))

    if not P is None:
        # *PERIODICPAIRS NPAIRS
        # [VIX VIX]_rows
        s += "*PERIODICPAIRS %d\n" % len(P)
        for i in range(len(P)):
            s+= "%d %d\n" % (P[i][0], P[i][1])
        s += "\n"

    return s

def get_boundary_verts():
    pass

def filter_boundary_faces(F, boundary_verts):
    pass

In [13]:
def quads2tris(Q):
    tris = np.empty((len(Q)*2,3),dtype=np.int)
    for i in range(len(Q)):
        tris[2*i + 0] = [Q[i][j] for j in [0,1,2]]
        tris[2*i + 1] = [Q[i][j] for j in [0,2,3]]
    return tris

def quads2edges(Q):
    E = np.empty((len(Q)*4,2),dtype=np.int)
    for i in range(len(Q)):
        E[4*i + 0] = [Q[i][j] for j in [0,1]]
        E[4*i + 1] = [Q[i][j] for j in [1,2]]
        E[4*i + 2] = [Q[i][j] for j in [2,3]]
        E[4*i + 3] = [Q[i][j] for j in [3,0]]
    return E

def get_grid_quads(nrows, ncols):
    Q = []
    ix = lambda i,j: i * ncols + j
    for i in range(nrows-1):
        for j in range(ncols-1):
            Q.append([ix(i,j),ix(i,j+1),ix(i+1,j+1),ix(i+1,j)])
    Q = np.array(Q, dtype=np.int)
    return Q

def add_grid_plot(p, nrows, ncols, phi):
    Q = get_grid_quads(nrows, ncols)
    p.add_edges(phi, quads2edges(Q))

    
def add_axes(p, l=1, R=np.eye(3)):
    AX = np.eye(3) * l
    for i in range(3):
        V = np.stack([np.zeros(3),AX[i]],axis=0)
        E = np.array([[0,1]],dtype=np.int)
        clr = ["red","green","blue"][i]
        p.add_edges(V @ R.T, E, shading={"line_color": clr});
        
Raxes = np.array([[1,0,0],
                  [0,0,1],
                  [0,-1,0]]) # xi1 -> x, x2 -> -z, h -> y
# Vni @ Rji

In [41]:
np.random.seed(1991)
V, H, P = generate_hex_grid(DX=(0.1,0.1,0.1),N=(6,4,2),center=True)
material_types = [0, 1]
M = np.random.choice(material_types,size=(6,2,4)).reshape(-1)
T, T2H = hexs2tets(H, diagmode=0)
M = M[T2H] # get tet material ids from associated hex ids

# collect set of tets with same material
S = []
for i in range(len(material_types)):
    S.append(sorted(np.where(M == material_types[i])[0]))
    
# define materials
M = [[1000, 1e6, 0.4], [2000, 5e6, 0.4]]
M = [[1000, 1e6, 0.4], [1000, 1e6, 0.4]]

# F = tets2tris(T)
# p = mp.plot(V @ Raxes.T, F, shading={"wireframe": True}, return_plot=True)
# add_axes(p, l=0.5, R=Raxes)

In [42]:
filestr = vega_file_string(V,T,M,S,P)
with open("../meshes/testmesh.veg",'w') as file:
    file.write(filestr)

In [ ]:

V = np.random.uniform(0,1,(10,3))
T = np.array([[0,1,2,3],[3,4,5,1]])
S = np.array([[0,1,8],[1,2,3,4,5,6,7,8,9,10,11,12,14,15,1,5,3,2,6,8,6,4,4,3,2,4,]])
M = np.array([[1000, 1e6, 0.4],[500, 2e6, 0.4]])
P =  np.array([[0,10],[3,4]])

print(vega_file_string(V,T,M,S,P=P))
# py define material
# py define voxel material ID

In [ ]:
V = np.array([
    [0,0,0],
    [0,0,1],
    [0,1,1],
    [0,1,0],
    [1,0,0],
    [1,0,1],
    [1,1,1],
    [1,1,0],
], dtype = np.float)
H = np.array([[0,1,2,3,4,5,6,7]],dtype=np.int)
T,T2H = hexs2tets(H, diagmode=0)
F = tets2tris(T)
F = tets2tris(T)
mp.plot(V, F, shading={"wireframe": True})

In [ ]:
V = np.random.uniform(-1,1,(4,3))*1
T = np.array([[0,1,2,3]],dtype=np.int)
F = tets2tris(T)
mp.plot(V, F, shading={"wireframe": True})

In [ ]:
nrows, ncols = 52,42
V = np.array([[-2.18522, -2.04333, 0.71984], [-2.06174, -1.9992, 0.738753], [-1.93457, -1.95916, 0.754706], [-1.80455, -1.9226, 0.766821], [-1.67234, -1.88903, 0.774463], [-1.53852, -1.85802, 0.777149], [-1.40362, -1.82916, 0.7745], [-1.26817, -1.80207, 0.766223], [-1.13264, -1.77638, 0.75209], [-0.997524, -1.75172, 0.73194], [-0.863272, -1.72774, 0.705665], [-0.730338, -1.70409, 0.673208], [-0.599158, -1.68044, 0.634564], [-0.470158, -1.65645, 0.589772], [-0.343748, -1.63181, 0.538914], [-0.220324, -1.60622, 0.482115], [-0.100266, -1.57937, 0.419537], [0.0160654, -1.55098, 0.351379], [0.128326, -1.52079, 0.277877], [0.236193, -1.48854, 0.199296], [0.339365, -1.454, 0.115933], [0.437564, -1.41696, 0.0281124], [0.530536, -1.37721, -0.0638181], [0.618049, -1.33458, -0.159487], [0.6999, -1.28892, -0.258504], [0.775911, -1.2401, -0.360461], [0.845931, -1.188, -0.464938], [0.909836, -1.13255, -0.571501], [0.96753, -1.07369, -0.679711], [1.01894, -1.01139, -0.789121], [1.06403, -0.945648, -0.899283], [1.10279, -0.8765, -1.00975], [1.13521, -0.804002, -1.12008], [1.16134, -0.728247, -1.22984], [1.18123, -0.649364, -1.33859], [1.19497, -0.567518, -1.44593], [1.20263, -0.48292, -1.55146], [1.20434, -0.395827, -1.65481], [1.20021, -0.306563, -1.75561], [1.19035, -0.215529, -1.85354], [1.17486, -0.123252, -1.9483], [1.15377, -0.030467, -2.03964], [-2.13355, -1.98849, 0.65261], [-2.0116, -1.94299, 0.672324], [-1.8863, -1.90129, 0.688812], [-1.75828, -1.86294, 0.701434], [-1.62812, -1.82751, 0.709649], [-1.49634, -1.7946, 0.713015], [-1.36346, -1.76384, 0.71117], [-1.22997, -1.73484, 0.70383], [-1.09634, -1.70725, 0.690771], [-0.963051, -1.68071, 0.671829], [-0.830542, -1.65488, 0.646893], [-0.699256, -1.62942, 0.615903], [-0.569624, -1.604, 0.578845], [-0.442063, -1.5783, 0.535752], [-0.316978, -1.552, 0.486698], [-0.194762, -1.52481, 0.431799], [-0.0757881, -1.49643, 0.371206], [0.0395838, -1.46658, 0.305112], [0.151013, -1.43501, 0.233738], [0.25818, -1.40146, 0.157342], [0.360783, -1.3657, 0.0762099], [0.458544, -1.32751, -0.00934643], [0.551208, -1.28671, -0.0989885], [0.638545, -1.24311, -0.192356], [0.720349, -1.19657, -0.289069], [0.796439, -1.14695, -0.388731], [0.866662, -1.09414, -0.490931], [0.93089, -1.03806, -0.595246], [0.989023, -0.978659, -0.701245], [1.04099, -0.915897, -0.808492], [1.08674, -0.849774, -0.916547], [1.12625, -0.780314, -1.02497], [1.15954, -0.707574, -1.13333], [1.18662, -0.631641, -1.2412], [1.20754, -0.552636, -1.34815], [1.22239, -0.470717, -1.45378], [1.23126, -0.38608, -1.55769], [1.23423, -0.298969, -1.65952], [1.23145, -0.20968, -1.75891], [1.22301, -0.11857, -1.85552], [1.20905, -0.0260723, -1.94907], [1.18967, 0.0673046, -2.03929], [-2.08414, -1.93032, 0.585543], [-1.96334, -1.88377, 0.606523], [-1.83953, -1.84074, 0.624003], [-1.71318, -1.8009, 0.637509], [-1.58477, -1.76387, 0.646598], [-1.45478, -1.72931, 0.650883], [-1.32369, -1.69685, 0.650034], [-1.19197, -1.66613, 0.643784], [-1.06006, -1.63682, 0.631919], [-0.928431, -1.60857, 0.614278], [-0.797506, -1.58105, 0.590752], [-0.667721, -1.55392, 0.561278], [-0.539498, -1.52687, 0.525839], [-0.413248, -1.49958, 0.484463], [-0.289369, -1.47174, 0.437216], [-0.168246, -1.44306, 0.384208], [-0.0502514, -1.41325, 0.325584], [0.0642602, -1.38204, 0.261525], [0.174951, -1.34916, 0.192246], [0.281502, -1.31437, 0.117995], [0.383615, -1.27744, 0.0390468], [0.481012, -1.23816, -0.0442938], [0.573438, -1.19634, -0.131699], [0.660661, -1.1518, -0.222819], [0.742475, -1.10439, -0.317282], [0.818697, -1.05398, -0.414702], [0.889172, -1.00046, -0.514676], [0.953769, -0.943737, -0.616792], [1.01239, -0.883762, -0.720627], [1.06494, -0.820497, -0.825753], [1.11139, -0.753935, -0.93174], [1.15171, -0.684097, -1.03815], [1.18589, -0.611032, -1.14457], [1.21397, -0.53482, -1.25056], [1.23598, -0.455573, -1.35572], [1.25201, -0.373435, -1.45964], [1.26213, -0.288589, -1.56193], [1.26645, -0.201252, -1.66223], [1.2651, -0.111683, -1.76017], [1.25821, -0.0201833, -1.85544], [1.24592, 0.0729114, -1.94773], [1.2284, 0.167252, -2.03676], [-2.0364, -1.86925, 0.519212], [-1.91658, -1.82183, 0.541686], [-1.79402, -1.77771, 0.560443], [-1.66909, -1.73661, 0.575109], [-1.54221, -1.69821, 0.585315], [-1.4138, -1.66219, 0.590725], [-1.28431, -1.62822, 0.591045], [-1.15416, -1.59598, 0.586026], [-1.02381, -1.56512, 0.575469], [-0.893676, -1.53532, 0.559221], [-0.764189, -1.50625, 0.537174], [-0.635767, -1.4776, 0.509268], [-0.508823, -1.44905, 0.475483], [-0.38376, -1.42029, 0.435844], [-0.260971, -1.39102, 0.390414], [-0.140834, -1.36096, 0.339295], [-0.0237164, -1.32981, 0.282626], [0.0900303, -1.29731, 0.220582], [0.200072, -1.2632, 0.153371], [0.306091, -1.22724, 0.0812305], [0.407792, -1.1892, 0.00442955], [0.504896, -1.14888, -0.0767372], [0.597151, -1.10608, -0.16195], [0.684323, -1.06063, -0.250867], [0.766205, -1.01237, -0.343127], [0.842614, -0.961178, -0.438351], [0.913392, -0.906936, -0.536146], [0.978406, -0.849561, -0.636106], [1.03755, -0.788991, -0.737818], [1.09075, -0.725188, -0.84086], [1.13794, -0.65814, -0.94481], [1.1791, -0.587863, -1.04924], [1.21423, -0.514399, -1.15374], [1.24334, -0.437819, -1.25787], [1.26649, -0.358224, -1.36125], [1.28374, -0.275745, -1.46346], [1.29518, -0.190542, -1.56412], [1.30092, -0.102808, -1.66287], [1.30109, -0.0127644, -1.75935], [1.29583, 0.079341, -1.85324], [1.28531, 0.173246, -1.94423], [1.26973, 0.268697, -2.03202], [-1.98993, -1.8056, 0.453987], [-1.87101, -1.75743, 0.478084], [-1.74955, -1.71238, 0.498303], [-1.62586, -1.6702, 0.514328], [-1.50032, -1.63061, 0.525842], [-1.37332, -1.59332, 0.532549], [-1.24525, -1.55803, 0.534185], [-1.11653, -1.52442, 0.530524], [-0.987571, -1.49217, 0.521379], [-0.858793, -1.46096, 0.506608], [-0.730606, -1.4305, 0.486108], [-0.603418, -1.40046, 0.459821], [-0.477631, -1.37053, 0.427727], [-0.35364, -1.34042, 0.389848], [-0.23183, -1.30983, 0.346246], [-0.112576, -1.27848, 0.297018], [0.00376223, -1.24608, 0.242297], [0.116836, -1.21238, 0.182254], [0.226314, -1.17712, 0.117088], [0.331883, -1.14006, 0.0470311], [0.433247, -1.10097, -0.0276549], [0.530131, -1.05965, -0.106683], [0.62228, -1.01591, -0.189742], [0.709463, -0.969579, -0.276496], [0.791472, -0.920494, -0.366594], [0.868122, -0.868529, -0.459662], [0.939254, -0.813571, -0.555316], [1.00473, -0.755534, -0.653159], [1.06445, -0.694351, -0.752782], [1.11833, -0.629981, -0.853774], [1.16631, -0.562408, -0.955716], [1.20836, -0.491641, -1.05819], [1.24447, -0.417715, -1.16078], [1.27467, -0.340693, -1.26308], [1.29901, -0.260661, -1.36467], [1.31753, -0.177736, -1.46518], [1.33035, -0.0920594, -1.56421], [1.33757, -0.00379712, -1.66139], [1.33932, 0.086862, -1.75639], [1.33576, 0.179713, -1.84887], [1.32707, 0.274547, -1.93852], [1.31347, 0.371164, -2.02506], [-1.94444, -1.73959, 0.390129], [-1.82637, -1.69077, 0.415933], [-1.70591, -1.64492, 0.437741], [-1.58334, -1.60181, 0.45527], [-1.45901, -1.56119, 0.468238], [-1.33326, -1.5228, 0.47638], [-1.20647, -1.48633, 0.479458], [-1.07903, -1.4515, 0.477264], [-0.951331, -1.41801, 0.469625], [-0.823778, -1.38555, 0.456408], [-0.696765, -1.35381, 0.437517], [-0.570691, -1.3225, 0.412897], [-0.445946, -1.29132, 0.38253], [-0.322919, -1.25997, 0.346437], [-0.201986, -1.22817, 0.304676], [-0.0835152, -1.19563, 0.257343], [0.0321359, -1.16207, 0.204567], [0.144625, -1.12725, 0.146513], [0.253623, -1.09091, 0.0833749], [0.35882, -1.05281, 0.0153798], [0.459921, -1.01273, -0.0572182], [0.556654, -0.970469, -0.134138], [0.648763, -0.92583, -0.215075], [0.736019, -0.878643, -0.299702], [0.818212, -0.828757, -0.387671], [0.895157, -0.776036, -0.47862], [0.966695, -0.720369, -0.572168], [1.03269, -0.661666, -0.667926], [1.09303, -0.599858, -0.765493], [1.14763, -0.5349, -0.864461], [1.19644, -0.466771, -0.964421], [1.23942, -0.395474, -1.06496], [1.27657, -0.321035, -1.16566], [1.3079, -0.243508, -1.26613], [1.33346, -0.162968, -1.36595], [1.35332, -0.0795156, -1.46475], [1.36756, 0.00672572, -1.56214], [1.37631, 0.0956125, -1.65775], [1.3797, 0.186984, -1.75125], [1.37789, 0.280668, -1.8423], [1.37107, 0.376491, -1.93059], [1.35946, 0.474287, -2.01584], [-1.89971, -1.67141, 0.327837], [-1.78246, -1.62202, 0.355407], [-1.66293, -1.57548, 0.378894], [-1.54139, -1.53157, 0.398034], [-1.41815, -1.49006, 0.41257], [-1.29354, -1.45069, 0.422259], [-1.16791, -1.4132, 0.426882], [-1.04162, -1.37729, 0.426247], [-0.915067, -1.34269, 0.420196], [-0.788621, -1.3091, 0.408602], [-0.662669, -1.27622, 0.391378], [-0.537598, -1.24377, 0.36847], [-0.413791, -1.21144, 0.339863], [-0.291624, -1.17896, 0.305579], [-0.171471, -1.14604, 0.265675], [-0.0536918, -1.1124, 0.220244], [0.0613611, -1.07778, 0.169411], [0.173349, -1.04192, 0.113338], [0.281947, -1.00457, 0.0522146], [0.386848, -0.9655, -0.0137378], [0.487758, -0.924487, -0.0842703], [0.584406, -0.881327, -0.159108], [0.67654, -0.835832, -0.237951], [0.763929, -0.78783, -0.32048], [0.846363, -0.737167, -0.406353], [0.923658, -0.68371, -0.495212], [0.995653, -0.627345, -0.586685], [1.06221, -0.567978, -0.680388], [1.12322, -0.505539, -0.775925], [1.1786, -0.439979, -0.872896], [1.22828, -0.371271, -0.970894], [1.27224, -0.299412, -1.06951], [1.31046, -0.224421, -1.16835], [1.34296, -0.146341, -1.26699], [1.3698, -0.0652376, -1.36505], [1.39102, 0.0188034, -1.46214], [1.40673, 0.105675, -1.55788], [1.41705, 0.195253, -1.65192], [1.42212, 0.287399, -1.7439], [1.4221, 0.381967, -1.83349], [1.41719, 0.478805, -1.9204], [1.4076, 0.57777, -2.00433], [-1.85556, -1.60123, 0.267265], [-1.73911, -1.55135, 0.296648], [-1.62047, -1.50421, 0.32188], [-1.49989, -1.45961, 0.342713], [-1.37765, -1.41732, 0.358905], [-1.25408, -1.37711, 0.370229], [-1.12951, -1.33871, 0.376483], [-1.00428, -1.30185, 0.377487], [-0.878754, -1.26627, 0.373092], [-0.753309, -1.23167, 0.363182], [-0.628316, -1.19777, 0.347675], [-0.504147, -1.16428, 0.32652], [-0.381179, -1.13092, 0.299706], [-0.25978, -1.09741, 0.267254], [-0.140315, -1.06347, 0.229221], [-0.0231396, -1.02883, 0.185698], [0.0913988, -0.993222, 0.136809], [0.202966, -0.956398, 0.0827111], [0.311241, -0.918113, 0.0235912], [0.415917, -0.878134, -0.040334], [0.516706, -0.836245, -0.10882], [0.613336, -0.79224, -0.181598], [0.705556, -0.745933, -0.258373], [0.793137, -0.697152, -0.338829], [0.875869, -0.645743, -0.422633], [0.953566, -0.591571, -0.50943], [1.02607, -0.534522, -0.598856], [1.09324, -0.474499, -0.690529], [1.15496, -0.41143, -0.784061], [1.21116, -0.34526, -0.879056], [1.26176, -0.275958, -0.975113], [1.30673, -0.203515, -1.07183], [1.34607, -0.127944, -1.1688], [1.37979, -0.0492772, -1.26564], [1.40793, 0.0324298, -1.36194], [1.43057, 0.117103, -1.45732], [1.44779, 0.20465, -1.55141], [1.45972, 0.294962, -1.64385], [1.4665, 0.38792, -1.7343], [1.4683, 0.483393, -1.82243], [1.46532, 0.581251, -1.90793], [1.45777, 0.681362, -1.99052], [-1.81184, -1.52919, 0.208543], [-1.69618, -1.47888, 0.239773], [-1.5784, -1.43123, 0.266803], [-1.45872, -1.38603, 0.289391], [-1.33743, -1.34307, 0.307308], [-1.21482, -1.30213, 0.32034], [-1.09121, -1.26294, 0.328293], [-0.966945, -1.22525, 0.331001], [-0.842365, -1.1888, 0.328323], [-0.717829, -1.1533, 0.320149], [-0.5937, -1.11849, 0.306402], [-0.470343, -1.08407, 0.287037], [-0.348123, -1.04978, 0.262044], [-0.227404, -1.01534, 0.231444], [-0.108542, -0.980476, 0.195296], [0.00811077, -0.944919, 0.153689], [0.122214, -0.908411, 0.106744], [0.233436, -0.870702, 0.0546176], [0.34146, -0.831551, -0.00250792], [0.445982, -0.79073, -0.0644193], [0.546716, -0.748022, -0.130876], [0.643391, -0.703224, -0.201614], [0.735759, -0.65615, -0.276342], [0.823589, -0.606629, -0.35475], [0.906673, -0.554506, -0.436508], [0.984826, -0.499646, -0.521269], [1.05789, -0.441931, -0.60867], [1.12571, -0.381265, -0.698337], [1.1882, -0.317571, -0.789887], [1.24525, -0.25079, -0.882926], [1.29681, -0.180889, -0.97706], [1.34285, -0.10785, -1.07189], [1.38334, -0.03168, -1.16702], [1.41831, 0.0475955, -1.26204], [1.44781, 0.129931, -1.35659], [1.47189, 0.215264, -1.45026], [1.49066, 0.303513, -1.5427], [1.50423, 0.394584, -1.63354], [1.51275, 0.48837, -1.72245], [1.5164, 0.584755, -1.80909], [1.51536, 0.683621, -1.89317], [1.50986, 0.784846, -1.97439], [-1.76843, -1.4554, 0.151777], [-1.65356, -1.40474, 0.184883], [-1.53661, -1.35665, 0.213751], [-1.4178, -1.31095, 0.238145], [-1.29739, -1.26742, 0.257842], [-1.17568, -1.22584, 0.272638], [-1.05298, -1.18597, 0.282348], [-0.929593, -1.14756, 0.286814], [-0.805873, -1.11034, 0.285902], [-0.682163, -1.07406, 0.279508], [-0.558815, -1.03843, 0.26756], [-0.436187, -1.00319, 0.250016], [-0.314634, -0.968072, 0.226868], [-0.194513, -0.932794, 0.19814], [-0.0761763, -0.897092, 0.163889], [0.0400319, -0.860704, 0.124204], [0.153774, -0.823374, 0.0792053], [0.264724, -0.784856, 0.0290458], [0.372566, -0.74491, -0.0260936], [0.477001, -0.70331, -0.086003], [0.577743, -0.659842, -0.150446], [0.674525, -0.614304, -0.21916], [0.767098, -0.56651, -0.291861], [0.855234, -0.516289, -0.368242], [0.938724, -0.463487, -0.447977], [1.01738, -0.407967, -0.530723], [1.09105, -0.349611, -0.616122], [1.15958, -0.288319, -0.703805], [1.22287, -0.224011, -0.793392], [1.28083, -0.156627, -0.884494], [1.33339, -0.0861256, -0.97672], [1.38052, -0.0124872, -1.06968], [1.42221, 0.0642891, -1.16297], [1.45848, 0.144185, -1.2562], [1.48936, 0.227163, -1.34899], [1.51492, 0.313169, -1.44096], [1.53526, 0.402134, -1.53174], [1.55051, 0.493971, -1.62098], [1.5608, 0.588587, -1.70833], [1.56631, 0.685876, -1.79347], [1.56723, 0.785727, -1.8761], [1.56379, 0.888028, -1.95593], [-1.72524, -1.37999, 0.0970617], [-1.61116, -1.32903, 0.132067], [-1.49502, -1.28058, 0.162805], [-1.37705, -1.23446, 0.189043], [-1.25749, -1.19045, 0.210564], [-1.13662, -1.14833, 0.22717], [-1.01475, -1.10788, 0.238684], [-0.892185, -1.06884, 0.244951], [-0.769255, -1.03096, 0.245846], [-0.646298, -0.99399, 0.24127], [-0.523656, -0.957654, 0.231153], [-0.40168, -0.92169, 0.215457], [-0.280719, -0.885828, 0.194176], [-0.161122, -0.849804, 0.167335], [-0.0432364, -0.813353, 0.134991], [0.0725988, -0.776217, 0.0972336], [0.18605, -0.738144, 0.0541818], [0.296795, -0.69889, 0.005986], [0.404522, -0.658218, -0.0471747], [0.508933, -0.615904, -0.105093], [0.609745, -0.571735, -0.167535], [0.706693, -0.52551, -0.234243], [0.799529, -0.477045, -0.304935], [0.888024, -0.426168, -0.379308], [0.971971, -0.372724, -0.45704], [1.05119, -0.316577, -0.537792], [1.1255, -0.257605, -0.62121], [1.19479, -0.195709, -0.706928], [1.25893, -0.130805, -0.79457], [1.31783, -0.0628283, -0.883753], [1.37143, 0.00826425, -0.974087], [1.4197, 0.0824982, -1.06518], [1.46262, 0.15988, -1.15664], [1.50021, 0.240397, -1.24809], [1.53251, 0.324021, -1.33913], [1.55958, 0.410703, -1.4294], [1.58153, 0.500382, -1.51852], [1.59847, 0.592984, -1.60614], [1.61055, 0.68842, -1.69193], [1.61794, 0.786593, -1.77556], [1.62084, 0.887401, -1.85673], [1.61947, 0.990735, -1.93515], [-1.68218, -1.30305, 0.0444779], [-1.5689, -1.25186, 0.0814021], [-1.45356, -1.20313, 0.114036], [-1.3364, -1.15666, 0.142149], [-1.21765, -1.11225, 0.165529], [-1.09758, -1.06968, 0.183982], [-0.976495, -1.02874, 0.197336], [-0.854691, -0.989168, 0.205441], [-0.732489, -0.950727, 0.208177], [-0.610219, -0.913161, 0.205448], [-0.488216, -0.876209, 0.197189], [-0.366824, -0.839609, 0.183363], [-0.246386, -0.803099, 0.163967], [-0.127244, -0.766415, 0.139026], [-0.00974109, -0.7293, 0.108598], [0.105788, -0.691497, 0.0727722], [0.219015, -0.652758, 0.0316667], [0.32962, -0.61284, -0.0145695], [0.437294, -0.57151, -0.0657589], [0.541742, -0.528546, -0.121697], [0.642683, -0.483735, -0.182152], [0.739853, -0.436878, -0.246869], [0.833005, -0.387791, -0.315569], [0.921912, -0.336303, -0.387952], [1.00637, -0.282258, -0.4637], [1.08619, -0.225518, -0.542477], [1.1612, -0.165963, -0.623933], [1.23129, -0.103488, -0.707705], [1.29631, -0.0380094, -0.793421], [1.3562, 0.0305409, -0.880699], [1.41088, 0.10221, -0.969156], [1.46032, 0.177029, -1.0584], [1.50451, 0.255007, -1.14805], [1.54345, 0.336139, -1.23771], [1.5772, 0.4204, -1.32701], [1.60581, 0.507751, -1.41557], [1.62939, 0.598137, -1.50303], [1.64805, 0.691489, -1.58904], [1.66194, 0.787726, -1.67325], [1.67123, 0.886758, -1.75536], [1.67611, 0.988487, -1.83505], [1.67681, 1.09281, -1.91204], [-1.6392, -1.22468, -0.00590154], [-1.52671, -1.17333, 0.0329569], [-1.41217, -1.12438, 0.0675068], [-1.29579, -1.07764, 0.0975201], [-1.17782, -1.03291, 0.122787], [-1.05852, -0.989981, 0.143116], [-0.938178, -0.948629, 0.158339], [-0.817084, -0.908617, 0.168313], [-0.695556, -0.869701, 0.172916], [-0.573914, -0.831632, 0.17206], [-0.452491, -0.794153, 0.165679], [-0.33162, -0.757006, 0.153742], [-0.211641, -0.719934, 0.136244], [-0.0928915, -0.682676, 0.113213], [0.0242924, -0.644978, 0.0847073], [0.139579, -0.606587, 0.050815], [0.252643, -0.567257, 0.0116545], [0.363168, -0.526747, -0.0326268], [0.470849, -0.484827, -0.081853], [0.575392, -0.441276, -0.135821], [0.676518, -0.395883, -0.194302], [0.773965, -0.34845, -0.257043], [0.867486, -0.298792, -0.323767], [0.956855, -0.246739, -0.394178], [1.04187, -0.192135, -0.467961], [1.12234, -0.134842, -0.544782], [1.1981, -0.074737, -0.624294], [1.26902, -0.0117145, -0.706138], [1.33498, 0.0543125, -0.789944], [1.39589, 0.123414, -0.875335], [1.45169, 0.19564, -0.961928], [1.50233, 0.271025, -1.04934], [1.54782, 0.349584, -1.13718], [1.58815, 0.431316, -1.22507], [1.62337, 0.516201, -1.31263], [1.65355, 0.604205, -1.39949], [1.67878, 0.695279, -1.48528], [1.69918, 0.78936, -1.56967], [1.7149, 0.886373, -1.6523], [1.72609, 0.986232, -1.73287], [1.73297, 1.08884, -1.81107], [1.73575, 1.1941, -1.88662], [-1.59623, -1.14498, -0.0540107], [-1.48454, -1.09352, -0.0132055], [-1.37079, -1.04442, 0.023277], [-1.25519, -0.997481, 0.0552098], [-1.13798, -0.952512, 0.0823845], [-1.01941, -0.909301, 0.104613], [-0.89977, -0.867626, 0.12173], [-0.779343, -0.827257, 0.133593], [-0.65844, -0.787953, 0.140086], [-0.537376, -0.749466, 0.141121], [-0.416476, -0.711546, 0.136636], [-0.296071, -0.673939, 0.1266], [-0.176493, -0.636387, 0.111011], [-0.0580762, -0.598638, 0.0898976], [0.0588474, -0.560436, 0.0633177], [0.17395, -0.521533, 0.0313595], [0.28691, -0.481686, -0.00585902], [0.397413, -0.440655, -0.0481913], [0.505157, -0.398213, -0.0954628], [0.60985, -0.354139, -0.147472], [0.711215, -0.308224, -0.203992], [0.80899, -0.26027, -0.264772], [0.902931, -0.210094, -0.329537], [0.992812, -0.157525, -0.397994], [1.07843, -0.102408, -0.469829], [1.15959, -0.044602, -0.544712], [1.23614, 0.0160158, -0.622299], [1.30794, 0.0795521, -0.702233], [1.37487, 0.146096, -0.784147], [1.43684, 0.21572, -0.867666], [1.49379, 0.288478, -0.95241], [1.54568, 0.364407, -1.038], [1.59249, 0.443525, -1.12404], [1.63424, 0.525835, -1.21017], [1.67097, 0.611322, -1.29599], [1.70274, 0.699957, -1.38115], [1.72964, 0.791695, -1.46528], [1.7518, 0.886478, -1.54804], [1.76935, 0.984234, -1.62909], [1.78247, 1.08488, -1.70811], [1.79135, 1.18833, -1.78481], [1.7962, 1.29448, -1.8589], [-1.55323, -1.06403, -0.0997897], [-1.44236, -1.01253, -0.0570277], [-1.32939, -0.963343, -0.0186], [-1.21456, -0.916277, 0.0152672], [-1.09808, -0.87114, 0.0443667], [-0.980222, -0.827722, 0.0685122], [-0.861248, -0.785806, 0.0875398], [-0.74145, -0.74516, 0.10131], [-0.621129, -0.705549, 0.109709], [-0.500596, -0.666729, 0.112649], [-0.380171, -0.628451, 0.110073], [-0.260179, -0.590464, 0.101948], [-0.140949, -0.552516, 0.0882754], [-0.0228107, -0.514353, 0.0690832], [0.0939075, -0.475726, 0.0444301], [0.208881, -0.436387, 0.0144043], [0.321791, -0.396095, -0.0208769], [0.432327, -0.354613, -0.0612675], [0.540188, -0.311715, -0.106594], [0.645084, -0.267182, -0.156657], [0.746739, -0.220806, -0.211229], [0.844893, -0.172389, -0.270063], [0.939302, -0.121749, -0.332886], [1.02974, -0.068714, -0.399406], [1.116, -0.0131295, -0.469312], [1.19791, 0.0451454, -0.542277], [1.27529, 0.106235, -0.617957], [1.348, 0.170248, -0.695999], [1.41594, 0.237275, -0.776037], [1.47901, 0.307389, -0.8577], [1.53715, 0.380649, -0.940609], [1.59031, 0.457092, -1.02438], [1.63848, 0.536742, -1.10865], [1.68167, 0.619603, -1.19301], [1.71992, 0.705666, -1.27711], [1.7533, 0.794904, -1.36057], [1.7819, 0.887276, -1.44304], [1.80583, 0.982727, -1.52417], [1.82524, 1.08119, -1.60363], [1.84029, 1.18259, -1.6811], [1.85117, 1.28683, -1.75628], [1.85811, 1.39382, -1.8289], [-1.51017, -0.981914, -0.143183], [-1.40011, -0.930438, -0.098457], [-1.28794, -0.881229, -0.0580747], [-1.17386, -0.834105, -0.022262], [-1.05811, -0.788871, 0.00877459], [-0.940929, -0.745322, 0.0348499], [-0.822596, -0.70324, 0.0558016], [-0.703393, -0.662397, 0.0714917], [-0.583616, -0.62256, 0.0818077], [-0.463571, -0.583486, 0.0866642], [-0.343575, -0.54493, 0.0860036], [-0.223949, -0.506644, 0.0797967], [-0.105018, -0.468377, 0.0680437], [0.012892, -0.429879, 0.0507738], [0.129456, -0.390902, 0.0280461], [0.244353, -0.351201, -5.14453e-05], [0.357265, -0.310536, -0.0334017], [0.467885, -0.268673, -0.0718596], [0.575914, -0.225386, -0.115252], [0.681062, -0.180458, -0.163381], [0.783057, -0.13368, -0.216022], [0.881637, -0.0848587, -0.272926], [0.976562, -0.0338088, -0.333824], [1.0676, 0.0196394, -0.398426], [1.15456, 0.0756421, -0.466422], [1.23724, 0.13434, -0.537486], [1.31549, 0.195859, -0.611278], [1.38917, 0.260307, -0.687446], [1.45815, 0.327778, -0.765627], [1.52235, 0.398348, -0.845451], [1.58171, 0.472074, -0.926541], [1.63617, 0.549, -1.00852], [1.68572, 0.629149, -1.09101], [1.73038, 0.71253, -1.17362], [1.77019, 0.799136, -1.256], [1.8052, 0.888943, -1.33777], [1.83551, 0.981915, -1.41858], [1.86123, 1.078, -1.49808], [1.8825, 1.17713, -1.57594], [1.89949, 1.27923, -1.65185], [1.91238, 1.38422, -1.72551], [1.9214, 1.49199, -1.79664], [-1.46702, -0.898718, -0.184139], [-1.35779, -0.847323, -0.137444], [-1.24641, -0.798158, -0.0951008], [-1.13308, -0.751043, -0.0573351], [-1.01803, -0.705784, -0.0243529], [-0.901518, -0.662176, 0.00366115], [-0.783801, -0.620003, 0.0265457], [-0.665163, -0.579039, 0.0441639], [-0.545895, -0.539052, 0.0564045], [-0.426301, -0.499803, 0.0631831], [-0.306693, -0.461048, 0.0644432], [-0.187389, -0.42254, 0.0601564], [-0.068711, -0.384031, 0.0503235], [0.0490183, -0.345273, 0.0349743], [0.165477, -0.306021, 0.0141678], [0.280345, -0.266031, -0.0120077], [0.393309, -0.225064, -0.0434358], [0.504062, -0.182889, -0.0799716], [0.612307, -0.139279, -0.121443], [0.717756, -0.0940195, -0.167653], [0.820137, -0.0469033, -0.218377], [0.919191, 0.00226491, -0.273369], [1.01468, 0.0536682, -0.332361], [1.10636, 0.107477, -0.395063], [1.19405, 0.163847, -0.461169], [1.27756, 0.22292, -0.530353], [1.35672, 0.284822, -0.602277], [1.43139, 0.349663, -0.67659], [1.50145, 0.417537, -0.752932], [1.56682, 0.488522, -0.830934], [1.62742, 0.562677, -0.91022], [1.68321, 0.640048, -0.990416], [1.73418, 0.72066, -1.07114], [1.78034, 0.804525, -1.15202], [1.82171, 0.891638, -1.23269], [1.85838, 0.981979, -1.31278], [1.89041, 1.07551, -1.39193], [1.91793, 1.17219, -1.4698], [1.94107, 1.27194, -1.54606], [1.96001, 1.3747, -1.6204], [1.97492, 1.48038, -1.69252], [1.98602, 1.58889, -1.76215], [-1.42377, -0.81452, -0.22261], [-1.31537, -0.763265, -0.173943], [-1.20478, -0.714208, -0.129635], [-1.09219, -0.667169, -0.0899119], [-0.977845, -0.621954, -0.0549792], [-0.861979, -0.578359, -0.0250212], [-0.744856, -0.536168, -0.000198664], [-0.626755, -0.495157, 0.0193522], [-0.507965, -0.455096, 0.0335209], [-0.388786, -0.415747, 0.0422241], [-0.269529, -0.376868, 0.0454057], [-0.150507, -0.338214, 0.0430382], [-0.0320392, -0.29954, 0.0351228], [0.0855536, -0.260597, 0.0216896], [0.201952, -0.221143, 0.00279775], [0.316838, -0.180936, -0.0214647], [0.4299, -0.139737, -0.0509812], [0.540833, -0.0973173, -0.0856076], [0.64934, -0.0534517, -0.125173], [0.755137, -0.00792492, -0.169479], [0.85795, 0.0394686, -0.218304], [0.957521, 0.0889237, -0.271404], [1.05361, 0.140623, -0.328509], [1.14599, 0.194737, -0.389332], [1.23445, 0.251422, -0.453568], [1.31882, 0.31082, -0.520892], [1.39892, 0.373058, -0.590969], [1.47462, 0.438245, -0.663448], [1.5458, 0.506479, -0.737971], [1.61236, 0.577836, -0.814169], [1.67424, 0.65238, -0.89167], [1.73139, 0.730155, -0.970098], [1.7838, 0.811191, -1.04908], [1.83147, 0.895501, -1.12823], [1.87444, 0.983081, -1.2072], [1.91277, 1.07391, -1.28561], [1.94655, 1.16797, -1.3631], [1.97588, 1.26519, -1.43935], [2.00091, 1.36552, -1.51401], [2.0218, 1.4689, -1.58678], [2.03873, 1.57522, -1.65736], [2.0519, 1.6844, -1.72548], [-1.3804, -0.729396, -0.258551], [-1.27285, -0.678343, -0.207912], [-1.16305, -0.629457, -0.161637], [-1.0512, -0.58256, -0.119955], [-0.937541, -0.537456, -0.0830696], [-0.822307, -0.493943, -0.0511658], [-0.705758, -0.451806, -0.0244037], [-0.588169, -0.410822, -0.00291927], [-0.469829, -0.37076, 0.0131778], [-0.351034, -0.331386, 0.0238045], [-0.232091, -0.292458, 0.0289053], [-0.113313, -0.253733, 0.0284531], [0.0049842, -0.214966, 0.0224495], [0.122482, -0.175912, 0.0109248], [0.238863, -0.136328, -0.00606176], [0.353811, -0.0959745, -0.0284222], [0.467015, -0.0546144, -0.0560403], [0.578173, -0.012018, -0.0887721], [0.686988, 0.0320376, -0.126447], [0.793176, 0.0777671, -0.168868], [0.896464, 0.125376, -0.215814], [0.996596, 0.175057, -0.26704], [1.09333, 0.226995, -0.32228], [1.18644, 0.281359, -0.381246], [1.27572, 0.338305, -0.443634], [1.36098, 0.397976, -0.509122], [1.44207, 0.460499, -0.577374], [1.51883, 0.525985, -0.648041], [1.59116, 0.594531, -0.720765], [1.65895, 0.666216, -0.795179], [1.72214, 0.741103, -0.870913], [1.78067, 0.81924, -0.947592], [1.83454, 0.900658, -1.02484], [1.88375, 0.985369, -1.10229], [1.92834, 1.07337, -1.17956], [1.96835, 1.16466, -1.2563], [2.00388, 1.25918, -1.33215], [2.03504, 1.35691, -1.40677], [2.06196, 1.45777, -1.47983], [2.0848, 1.56171, -1.55103], [2.10375, 1.66863, -1.62006], [2.119, 1.77843, -1.68666], [-1.33691, -0.643421, -0.291918], [-1.23021, -0.59263, -0.239308], [-1.1212, -0.54398, -0.191069], [-1.0101, -0.497289, -0.147428], [-0.897118, -0.452364, -0.108592], [-0.782501, -0.409002, -0.074743], [-0.666508, -0.366989, -0.046043], [-0.54941, -0.326102, -0.0226272], [-0.431493, -0.286113, -0.0046049], [-0.313051, -0.246786, 0.00794101], [-0.19439, -0.207882, 0.0149555], [-0.0758203, -0.169159, 0.0164117], [0.0423442, -0.130373, 0.0123113], [0.159787, -0.0912804, 0.00268483], [0.276191, -0.0516387, -0.0124084], [0.391242, -0.011209, -0.0328806], [0.504632, 0.0302437, -0.0586157], [0.616057, 0.0729487, -0.08947], [0.725223, 0.117128, -0.125273], [0.831845, 0.162996, -0.165829], [0.935652, 0.210756, -0.210917], [1.03639, 0.260604, -0.260292], [1.1338, 0.312721, -0.31369], [1.22768, 0.367277, -0.370823], [1.31782, 0.424429, -0.431387], [1.40402, 0.48432, -0.495062], [1.48612, 0.547076, -0.561512], [1.56399, 0.612811, -0.630391], [1.63749, 0.68162, -0.701339], [1.70654, 0.753585, -0.773992], [1.77106, 0.828771, -0.84798], [1.83101, 0.907224, -0.922928], [1.88636, 0.988977, -0.998462], [1.93713, 1.07405, -1.07421], [1.98335, 1.16243, -1.1498], [2.02506, 1.25412, -1.22488], [2.06236, 1.34907, -1.29909], [2.09535, 1.44725, -1.3721], [2.12417, 1.5486, -1.44357], [2.14897, 1.65304, -1.51319], [2.16993, 1.7605, -1.58067], [2.18726, 1.87088, -1.64574], [-1.29331, -0.556671, -0.322672], [-1.18746, -0.506203, -0.268095], [-1.07926, -0.45785, -0.217894], [-0.968887, -0.41143, -0.172299], [-0.856578, -0.366751, -0.131514], [-0.742568, -0.323608, -0.0957247], [-0.627113, -0.281788, -0.0650914], [-0.510485, -0.241069, -0.0397493], [-0.392965, -0.201223, -0.0198081], [-0.274849, -0.162016, -0.00535016], [-0.156439, -0.123209, 0.00356948], [-0.0380436, -0.0845598, 0.0069241], [0.0800242, -0.0458265, 0.00471551], [0.197449, -0.00676605, -0.00302572], [0.313916, 0.0328628, -0.0162403], [0.429112, 0.0732983, -0.0348405], [0.542728, 0.114775, -0.0587104], [0.654462, 0.157521, -0.0877066], [0.76402, 0.201757, -0.121659], [0.871117, 0.247698, -0.160372], [0.975484, 0.295548, -0.203625], [1.07686, 0.3455, -0.251174], [1.17501, 0.397736, -0.302754], [1.26969, 0.452427, -0.358079], [1.36072, 0.509728, -0.416846], [1.44789, 0.569784, -0.478734], [1.53105, 0.63272, -0.543409], [1.61004, 0.698651, -0.610524], [1.68475, 0.767674, -0.679721], [1.75509, 0.83987, -0.750637], [1.82097, 0.915304, -0.8229], [1.88236, 0.994026, -0.896138], [1.93922, 1.07607, -0.969977], [1.99157, 1.16145, -1.04405], [2.03943, 1.25016, -1.11798], [2.08286, 1.3422, -1.19141], [2.12194, 1.43754, -1.26399], [2.15677, 1.53612, -1.33538], [2.18749, 1.6379, -1.40526], [2.21425, 1.74281, -1.4733], [2.23723, 1.85075, -1.53923], [2.25663, 1.96164, -1.60277], [-1.24959, -0.469217, -0.350774], [-1.14461, -0.419134, -0.294235], [-1.03721, -0.37114, -0.242079], [-0.927573, -0.325056, -0.194534], [-0.81593, -0.280688, -0.151808], [-0.702515, -0.237831, -0.114084], [-0.587584, -0.196273, -0.0815249], [-0.471405, -0.155792, -0.0542648], [-0.35426, -0.11616, -0.0324135], [-0.236443, -0.0771432, -0.0160537], [-0.118254, -0.0385035, -0.00524031], [0, 0, 0], [0.118006, 0.0386093, -0.000330971], [0.23545, 0.0775664, -0.00620268], [0.352017, 0.117112, -0.0175559], [0.467396, 0.157484, -0.0343029], [0.581278, 0.198915, -0.056328], [0.693362, 0.241635, -0.0834879], [0.803352, 0.285862, -0.115613], [0.910967, 0.331812, -0.152507], [1.01593, 0.379687, -0.19395], [1.11799, 0.429681, -0.2397], [1.2169, 0.481976, -0.28949], [1.31244, 0.536742, -0.343035], [1.40439, 0.594135, -0.400033], [1.49257, 0.654299, -0.460163], [1.57681, 0.717361, -0.523091], [1.65697, 0.783435, -0.588469], [1.73292, 0.852619, -0.655943], [1.80457, 0.924994, -0.725146], [1.87184, 1.00063, -0.79571], [1.93468, 1.07957, -0.867261], [1.99308, 1.16185, -0.939427], [2.04702, 1.24749, -1.01183], [2.09655, 1.33649, -1.08412], [2.1417, 1.42883, -1.15592], [2.18257, 1.5245, -1.22688], [2.21926, 1.62344, -1.29667], [2.25188, 1.7256, -1.36496], [2.28061, 1.83091, -1.43143], [2.3056, 1.93928, -1.4958], [2.32707, 2.05063, -1.5578], [-1.20577, -0.381134, -0.376188], [-1.10167, -0.331494, -0.317694], [-0.995071, -0.283923, -0.26359], [-0.88617, -0.238238, -0.214104], [-0.775186, -0.194246, -0.169445], [-0.662356, -0.151742, -0.129797], [-0.547933, -0.110514, -0.0953208], [-0.432185, -0.0703396, -0.0661534], [-0.315393, -0.030991, -0.0424039], [-0.197849, 0.00776519, -0.0241551], [-0.0798515, 0.0461668, -0.011462], [0.0382914, 0.084454, -0.00435151], [0.156269, 0.122869, -0.00282177], [0.273767, 0.161652, -0.00684234], [0.390472, 0.201045, -0.0163541], [0.506072, 0.241284, -0.0312694], [0.62026, 0.282602, -0.0514726], [0.732732, 0.325227, -0.0768206], [0.843195, 0.369379, -0.107144], [0.951366, 0.415272, -0.142246], [1.05697, 0.463108, -0.181908], [1.15975, 0.513081, -0.225887], [1.25946, 0.565374, -0.273917], [1.35588, 0.620155, -0.325713], [1.4488, 0.677581, -0.380972], [1.53802, 0.737796, -0.439374], [1.62338, 0.800928, -0.500585], [1.70473, 0.867091, -0.564258], [1.78195, 0.936382, -0.630037], [1.85495, 1.00888, -0.697557], [1.92363, 1.08466, -0.766448], [1.98796, 1.16377, -0.836338], [2.0479, 1.24624, -0.906854], [2.10346, 1.33209, -0.977623], [2.15466, 1.42132, -1.04828], [2.20156, 1.51392, -1.11846], [2.24423, 1.60987, -1.18782], [2.28277, 1.70911, -1.25602], [2.31731, 1.8116, -1.32272], [2.348, 1.91726, -1.38763], [2.37501, 2.02601, -1.45045], [2.39854, 2.13776, -1.5109], [-1.16186, -0.29249, -0.398878], [-1.05865, -0.243356, -0.338439], [-0.952864, -0.196269, -0.282397], [-0.844696, -0.151047, -0.23098], [-0.734366, -0.107496, -0.184399], [-0.62211, -0.0654115, -0.142837], [-0.508181, -0.0245801, -0.106458], [-0.392845, 0.0152196, -0.0753963], [-0.276384, 0.0542157, -0.0497629], [-0.159087, 0.0926417, -0.0296406], [-0.0412537, 0.130735, -0.0150846], [0.0768091, 0.168736, -0.00612195], [0.194791, 0.206886, -0.00275103], [0.312378, 0.245426, -0.0049415], [0.429257, 0.284596, -0.0126343], [0.545116, 0.324632, -0.025742], [0.659647, 0.365769, -0.0441489], [0.772547, 0.408232, -0.067712], [0.883523, 0.452242, -0.0962616], [0.992288, 0.498013, -0.129602], [1.09857, 0.545746, -0.167514], [1.20211, 0.595635, -0.209753], [1.30267, 0.647863, -0.256056], [1.4, 0.702598, -0.306136], [1.49392, 0.759999, -0.359689], [1.58421, 0.820207, -0.416397], [1.67073, 0.883351, -0.475925], [1.75331, 0.949546, -0.537925], [1.83183, 1.01889, -0.602041], [1.90619, 1.09146, -0.667909], [1.97631, 1.16734, -0.735157], [2.04214, 1.24656, -0.803414], [2.10365, 1.32916, -0.872306], [2.16085, 1.41517, -0.941461], [2.21375, 1.50458, -1.01051], [2.2624, 1.59739, -1.0791], [2.30687, 1.69356, -1.14687], [2.34727, 1.79305, -1.21348], [2.38373, 1.89582, -1.27862], [2.41638, 2.00178, -1.34196], [2.4454, 2.11085, -1.40323], [2.47099, 2.22295, -1.46214], [-1.1179, -0.203357, -0.418812], [-1.01558, -0.154788, -0.356439], [-0.910609, -0.108247, -0.29847], [-0.803171, -0.0635517, -0.245135], [-0.693489, -0.0205066, -0.196645], [-0.581797, 0.0210928, -0.153184], [-0.468348, 0.0614602, -0.114915], [-0.353407, 0.100817, -0.0819754], [-0.237255, 0.139393, -0.0544749], [-0.12018, 0.177419, -0.0324971], [-0.0024837, 0.215135, -0.0160975], [0.115529, 0.25278, -0.0053034], [0.233548, 0.290595, -0.000113442], [0.351259, 0.328822, -0.000497444], [0.468348, 0.3677, -0.00639651], [0.584503, 0.407465, -0.0177232], [0.699415, 0.448352, -0.0343621], [0.812782, 0.490585, -0.05617], [0.924309, 0.534387, -0.0829772], [1.03371, 0.579969, -0.114588], [1.14071, 0.627534, -0.150783], [1.24505, 0.677276, -0.191318], [1.34648, 0.729376, -0.235928], [1.44477, 0.784005, -0.284327], [1.53972, 0.841318, -0.336212], [1.63113, 0.90146, -0.391263], [1.71883, 0.964559, -0.449143], [1.80266, 1.03073, -0.509506], [1.88251, 1.10007, -0.571995], [1.95826, 1.17266, -0.636244], [2.02985, 1.24857, -0.701883], [2.09721, 1.32785, -0.768539], [2.16031, 1.41054, -0.835836], [2.21915, 1.49665, -0.903403], [2.27376, 1.58619, -0.970872], [2.32417, 1.67915, -1.03788], [2.37047, 1.7755, -1.10408], [2.41274, 1.87519, -1.16913], [2.45111, 1.97817, -1.23271], [2.48572, 2.08438, -1.2945], [2.51675, 2.19372, -1.35422], [2.54438, 2.30612, -1.41158], [-1.0739, -0.113805, -0.435958], [-0.97249, -0.0658609, -0.371664], [-0.86833, -0.0199278, -0.311782], [-0.761623, 0.0241785, -0.256544], [-0.652583, 0.0666534, -0.206159], [-0.541445, 0.107702, -0.160815], [-0.428461, 0.147539, -0.120674], [-0.313897, 0.186387, -0.0858737], [-0.198032, 0.224473, -0.0565253], [-0.0811553, 0.262032, -0.0327123], [0.0364327, 0.299301, -0.0144909], [0.154426, 0.33652, -0.00188855], [0.272514, 0.373931, 0.00509579], [0.390383, 0.411775, 0.00649204], [0.507718, 0.450291, 0.00235894], [0.624206, 0.489717, -0.0072162], [0.739538, 0.530285, -0.0221179], [0.853411, 0.572222, -0.0422031], [0.965527, 0.615747, -0.0673019], [1.0756, 0.661075, -0.0972186], [1.18336, 0.708407, -0.131733], [1.28853, 0.757937, -0.170601], [1.39088, 0.809847, -0.213557], [1.49017, 0.864306, -0.260315], [1.58618, 0.921472, -0.310571], [1.67873, 0.981488, -0.364003], [1.76765, 1.04448, -0.420276], [1.85277, 1.11057, -0.479041], [1.93397, 1.17985, -0.539941], [2.01115, 1.2524, -0.60261], [2.08422, 1.3283, -0.666675], [2.15313, 1.40758, -0.731763], [2.21784, 1.4903, -0.797498], [2.27835, 1.57646, -0.863507], [2.33468, 1.66608, -0.929422], [2.38686, 1.75913, -0.994882], [2.43498, 1.8556, -1.05953], [2.47913, 1.95544, -1.12304], [2.51942, 2.05859, -1.18507], [2.55599, 2.16498, -1.24531], [2.58903, 2.27455, -1.30349], [2.6187, 2.38718, -1.35931], [-1.02991, -0.0239006, -0.450286], [-0.929401, 0.0233577, -0.384085], [-0.826061, 0.0686216, -0.322306], [-0.720081, 0.112076, -0.26518], [-0.611677, 0.153917, -0.212919], [-0.501083, 0.19435, -0.165709], [-0.38855, 0.233591, -0.123715], [-0.274344, 0.271861, -0.0870751], [-0.158744, 0.30939, -0.0559001], [-0.0420403, 0.346413, -0.0302748], [0.0754669, 0.383166, -0.0102556], [0.193471, 0.419891, 0.00412943], [0.311662, 0.456828, 0.012881], [0.429723, 0.49422, 0.0160288], [0.54734, 0.532306, 0.0136312], [0.664199, 0.571323, 0.00577563], [0.77999, 0.611504, -0.00742261], [0.894406, 0.653075, -0.0258203], [1.00715, 0.696258, -0.0492475], [1.11794, 0.741264, -0.0775081], [1.22649, 0.788297, -0.110381], [1.33254, 0.837551, -0.147623], [1.43584, 0.889206, -0.188967], [1.53616, 0.943434, -0.234126], [1.63328, 1.00039, -0.282795], [1.72701, 1.06022, -0.334652], [1.81717, 1.12305, -0.389361], [1.90361, 1.189, -0.446572], [1.98619, 1.25816, -0.505925], [2.06482, 1.33061, -0.567054], [2.1394, 1.40644, -0.629585], [2.20988, 1.48567, -0.693144], [2.27622, 1.56836, -0.757353], [2.33841, 1.65453, -0.821838], [2.39647, 1.74416, -0.88623], [2.45044, 1.83726, -0.950166], [2.5004, 1.93379, -1.01329], [2.54642, 2.03372, -1.07527], [2.58863, 2.13698, -1.13577], [2.62716, 2.24352, -1.19449], [2.66219, 2.35324, -1.25112], [2.6939, 2.46606, -1.30541], [-0.985943, 0.0662877, -0.461766], [-0.886355, 0.112801, -0.393676], [-0.783836, 0.157334, -0.330017], [-0.678582, 0.200074, -0.271022], [-0.570807, 0.241217, -0.216903], [-0.460745, 0.280969, -0.167848], [-0.348647, 0.319548, -0.124022], [-0.234781, 0.357174, -0.0855643], [-0.119424, 0.394079, -0.0525863], [-0.00286738, 0.430497, -0.0251735], [0.114588, 0.466666, -0.0033829], [0.232635, 0.502827, 0.0127569], [0.350961, 0.539222, 0.0232462], [0.46925, 0.576092, 0.0281142], [0.587186, 0.613678, 0.0274193], [0.704454, 0.652217, 0.0212485], [0.820742, 0.691943, 0.00971726], [0.935742, 0.733081, -0.00703116], [1.04916, 0.775853, -0.0288265], [1.1607, 0.820471, -0.0554724], [1.27008, 0.86714, -0.0867475], [1.37704, 0.916051, -0.122407], [1.48133, 0.967388, -0.162184], [1.58272, 1.02132, -0.20579], [1.68098, 1.07801, -0.252919], [1.77592, 1.13759, -0.303248], [1.86737, 1.20019, -0.356439], [1.95515, 1.26594, -0.412142], [2.03915, 1.33492, -0.469995], [2.11925, 1.40723, -0.529629], [2.19537, 1.48292, -0.59067], [2.26743, 1.56205, -0.652741], [2.33542, 1.64466, -0.715464], [2.39931, 1.73076, -0.778462], [2.45912, 1.82037, -0.841365], [2.51489, 1.91345, -0.903808], [2.56668, 2.01, -0.965438], [2.61458, 2.10996, -1.02591], [2.65871, 2.21329, -1.0849], [2.69921, 2.31991, -1.1421], [2.73623, 2.42973, -1.19721], [2.76995, 2.54268, -1.24997], [-0.942056, 0.156694, -0.470371], [-0.843391, 0.202401, -0.400411], [-0.741696, 0.246143, -0.334891], [-0.637164, 0.288106, -0.274047], [-0.530011, 0.328488, -0.218091], [-0.420469, 0.367495, -0.167213], [-0.308792, 0.405345, -0.121578], [-0.195244, 0.442262, -0.0813268], [-0.0801067, 0.478475, -0.0465714], [0.0363289, 0.51422, -0.017398], [0.153762, 0.549736, 0.00613543], [0.271883, 0.585264, 0.0239998], [0.390379, 0.621047, 0.0361949], [0.508932, 0.657327, 0.0427496], [0.627226, 0.694344, 0.0437218], [0.744942, 0.732336, 0.0391984], [0.861766, 0.771537, 0.029295], [0.977392, 0.812173, 0.0141548], [1.09152, 0.854467, -0.00605159], [1.20385, 0.89863, -0.0311273], [1.31411, 0.944867, -0.0608503], [1.42202, 0.993371, -0.094975], [1.52735, 1.04432, -0.133233], [1.62984, 1.0979, -0.175336], [1.72928, 1.15425, -0.220976], [1.82546, 1.21352, -0.269828], [1.91822, 1.27584, -0.321553], [2.00738, 1.34132, -0.375798], [2.09282, 1.41007, -0.432201], [2.17442, 1.48217, -0.49039], [2.2521, 1.55767, -0.549989], [2.32578, 1.63664, -0.610619], [2.39542, 1.71912, -0.671899], [2.46103, 1.80511, -0.733451], [2.5226, 1.89462, -0.794904], [2.58017, 1.98764, -0.855889], [2.63381, 2.08415, -0.916053], [2.6836, 2.1841, -0.975051], [2.72965, 2.28743, -1.03255], [2.7721, 2.39407, -1.08825], [2.8111, 2.50396, -1.14185], [2.84684, 2.61698, -1.19308], [-0.898291, 0.247252, -0.476076], [-0.800556, 0.292095, -0.404264], [-0.699685, 0.334985, -0.336905], [-0.595872, 0.376109, -0.274234], [-0.489332, 0.415665, -0.216464], [-0.380299, 0.453864, -0.163787], [-0.269024, 0.49092, -0.116368], [-0.155775, 0.52706, -0.0743489], [-0.0408322, 0.562514, -0.0378435], [0.0755108, 0.597518, -0.00693854], [0.192952, 0.632313, 0.0183073], [0.311181, 0.667139, 0.037864], [0.429883, 0.70224, 0.0517308], [0.548739, 0.73786, 0.059936], [0.667428, 0.774239, 0.0625373], [0.785632, 0.811615, 0.0596214], [0.903036, 0.850222, 0.0513037], [1.01933, 0.890288, 0.0377277], [1.1342, 0.932034, 0.0190643], [1.24737, 0.975675, -0.00448904], [1.35855, 1.02141, -0.0327097], [1.46746, 1.06944, -0.0653508], [1.57385, 1.11995, -0.102143], [1.67748, 1.1731, -0.142797], [1.77814, 1.22905, -0.187002], [1.8756, 1.28795, -0.234433], [1.96971, 1.34993, -0.284747], [2.06028, 1.41509, -0.33759], [2.14719, 1.48354, -0.392597], [2.23032, 1.55537, -0.449395], [2.30957, 1.63063, -0.507605], [2.38489, 1.70938, -0.566844], [2.45622, 1.79166, -0.62673], [2.52355, 1.87748, -0.686883], [2.58689, 1.96685, -0.746927], [2.64628, 2.05976, -0.806494], [2.70177, 2.15617, -0.865225], [2.75345, 2.25604, -0.922777], [2.80142, 2.35933, -0.978817], [2.84582, 2.46595, -1.03303], [2.8868, 2.57584, -1.08514], [2.92454, 2.68888, -1.13485], [-0.8547, 0.337898, -0.478855], [-0.757901, 0.381818, -0.405214], [-0.657855, 0.423795, -0.336038], [-0.554756, 0.464018, -0.271563], [-0.448819, 0.502687, -0.212004], [-0.34028, 0.540011, -0.157552], [-0.229389, 0.576209, -0.108376], [-0.116416, 0.611506, -0.0646174], [-0.0016423, 0.646133, -0.0263912], [0.114638, 0.680329, 0.00621453], [0.23212, 0.714333, 0.0331402], [0.350492, 0.748389, 0.054355], [0.469437, 0.782739, 0.0698572], [0.588634, 0.817629, 0.0796748], [0.707761, 0.8533, 0.0838647], [0.826495, 0.889989, 0.0825136], [0.94452, 0.927933, 0.0757368], [1.06152, 0.96736, 0.0636779], [1.17719, 1.00849, 0.0465083], [1.29124, 1.05154, 0.024426], [1.40337, 1.09671, -0.00234534], [1.51332, 1.14421, -0.033558], [1.62082, 1.1942, -0.0689413], [1.72564, 1.24686, -0.108203], [1.82754, 1.30235, -0.151033], [1.92633, 1.36081, -0.197102], [2.02181, 1.42238, -0.246066], [2.11383, 1.48716, -0.297567], [2.20224, 1.55526, -0.351239], [2.28692, 1.62676, -0.406705], [2.36778, 1.70172, -0.463583], [2.44475, 1.7802, -0.521488], [2.51778, 1.86223, -0.580034], [2.58686, 1.94782, -0.638838], [2.65199, 2.037, -0.69752], [2.7132, 2.12973, -0.755711], [2.77055, 2.22599, -0.81305], [2.82412, 2.32574, -0.869189], [2.87401, 2.42893, -0.923795], [2.92035, 2.53548, -0.976555], [2.96331, 2.64531, -1.02717], [3.00304, 2.75832, -1.07538], [-0.81134, 0.42857, -0.478684], [-0.71548, 0.471507, -0.403239], [-0.616258, 0.512511, -0.33227], [-0.513868, 0.551772, -0.266016], [-0.408525, 0.58949, -0.204693], [-0.300463, 0.625877, -0.148495], [-0.189937, 0.66115, -0.0975891], [-0.0772156, 0.695538, -0.05212], [0.0374172, 0.729272, -0.0122037], [0.153665, 0.762591, 0.0220704], [0.271223, 0.795736, 0.0506418], [0.389775, 0.828951, 0.0734784], [0.509003, 0.862482, 0.0905779], [0.628583, 0.896572, 0.101967], [0.748189, 0.931464, 0.107703], [0.867499, 0.967398, 0.107872], [0.98619, 1.00461, 0.102588], [1.10395, 1.04333, 0.0919962], [1.22046, 1.08377, 0.0762679], [1.33543, 1.12616, 0.055602], [1.44856, 1.1707, 0.0302228], [1.55959, 1.21759, 0.00037962], [1.66825, 1.267, -0.0336554], [1.77429, 1.31911, -0.0715886], [1.87748, 1.37407, -0.113107], [1.97763, 1.43204, -0.157878], [2.07453, 1.49313, -0.205557], [2.16802, 1.55747, -0.255783], [2.25796, 1.62516, -0.308185], [2.34422, 1.69627, -0.362384], [2.42671, 1.77087, -0.417994], [2.50536, 1.84902, -0.474626], [2.58011, 1.93074, -0.531891], [2.65095, 2.01606, -0.589401], [2.71787, 2.10498, -0.646776], [2.78092, 2.19749, -0.703639], [2.84013, 2.29355, -0.759628], [2.89559, 2.39312, -0.814392], [2.9474, 2.49616, -0.867596], [2.99569, 2.60258, -0.918924], [3.0406, 2.7123, -0.968081], [3.08231, 2.82523, -1.01479], [-0.768272, 0.519205, -0.475543], [-0.673356, 0.561102, -0.398317], [-0.574956, 0.601073, -0.325582], [-0.473268, 0.63931, -0.257576], [-0.368505, 0.676015, -0.194517], [-0.260904, 0.7114, -0.136599], [-0.150721, 0.745684, -0.0839936], [-0.038225, 0.779096, -0.0368446], [0.076297, 0.811869, 0.00472945], [0.192547, 0.844243, 0.0406382], [0.310217, 0.876461, 0.0708194], [0.428989, 0.908767, 0.0952401], [0.548542, 0.941407, 0.113897], [0.668548, 0.974627, 0.126815], [0.78868, 1.00867, 0.134052], [0.90861, 1.04378, 0.135692], [1.02801, 1.08019, 0.131852], [1.14657, 1.11812, 0.122674], [1.26397, 1.15782, 0.108331], [1.37991, 1.19948, 0.0890232], [1.4941, 1.24332, 0.0649754], [1.60625, 1.28953, 0.0364385], [1.71611, 1.33829, 0.00368632], [1.82341, 1.38978, -0.0329853], [1.92794, 1.44415, -0.0732605], [2.02948, 1.50156, -0.116806], [2.12783, 1.56212, -0.163271], [2.22283, 1.62596, -0.212292], [2.31433, 1.69317, -0.263496], [2.4022, 1.76383, -0.316498], [2.48635, 1.83802, -0.370909], [2.5667, 1.91578, -0.426336], [2.64319, 1.99714, -0.482385], [2.7158, 2.08213, -0.538665], [2.78454, 2.17074, -0.594789], [2.84942, 2.26297, -0.650379], [2.9105, 2.35878, -0.705066], [2.96786, 2.45812, -0.758498], [3.02159, 2.56095, -0.810336], [3.07181, 2.66719, -0.860262], [3.11868, 2.77676, -0.907978], [3.16236, 2.88956, -0.953209], [-0.725563, 0.609745, -0.469411], [-0.631594, 0.650542, -0.390432], [-0.534015, 0.689421, -0.315957], [-0.433019, 0.726574, -0.246226], [-0.328823, 0.762203, -0.181459], [-0.221664, 0.796522, -0.121852], [-0.111799, 0.829752, -0.067577], [0.000500056, 0.862122, -0.0187801], [0.114944, 0.893867, 0.0244186], [0.231233, 0.925228, 0.0619268], [0.349054, 0.956449, 0.0936809], [0.468089, 0.987775, 0.119646], [0.588011, 1.01945, 0.139818], [0.70849, 1.05173, 0.154221], [0.829196, 1.08486, 0.162912], [0.949796, 1.11907, 0.165974], [1.06996, 1.1546, 0.163523], [1.18937, 1.19169, 0.155703], [1.30771, 1.23056, 0.142687], [1.42467, 1.27142, 0.124675], [1.53996, 1.31449, 0.101894], [1.65328, 1.35996, 0.0745956], [1.76438, 1.40801, 0.0430563], [1.873, 1.45881, 0.00757397], [1.9789, 1.51253, -0.0315327], [2.08187, 1.56931, -0.0739268], [2.18171, 1.62928, -0.119255], [2.27826, 1.69255, -0.16715], [2.37135, 1.75923, -0.217233], [2.46086, 1.82939, -0.269116], [2.54669, 1.9031, -0.322405], [2.62876, 1.98041, -0.376702], [2.70701, 2.06136, -0.431607], [2.78142, 2.14596, -0.486725], [2.85198, 2.23421, -0.541663], [2.91871, 2.32611, -0.596039], [2.98167, 2.42161, -0.649479], [3.04092, 2.52068, -0.701627], [3.09656, 2.62325, -0.75214], [3.14872, 2.72926, -0.800696], [3.19753, 2.83862, -0.846996], [3.24316, 2.95123, -0.890764], [-0.683288, 0.70013, -0.46027], [-0.590268, 0.739771, -0.379566], [-0.493505, 0.777499, -0.303379], [-0.393191, 0.813506, -0.231953], [-0.289546, 0.847997, -0.165508], [-0.182807, 0.881186, -0.104241], [-0.0732334, 0.913297, -0.0483276], [0.0389, 0.944559, 0.00208473], [0.153302, 0.975208, 0.0468738], [0.269669, 1.00549, 0.0859455], [0.387685, 1.03564, 0.119234], [0.507028, 1.06592, 0.146703], [0.627367, 1.09657, 0.168347], [0.748369, 1.12783, 0.184189], [0.869699, 1.15997, 0.194284], [0.991022, 1.19321, 0.198716], [1.11201, 1.2278, 0.197599], [1.23232, 1.26397, 0.191078], [1.35165, 1.30194, 0.179326], [1.46968, 1.34194, 0.162544], [1.58612, 1.38416, 0.140961], [1.70066, 1.42882, 0.114829], [1.81305, 1.47608, 0.0844275], [1.92303, 1.52614, 0.0500569], [2.03035, 1.57914, 0.0120388], [2.1348, 1.63523, -0.0292863], [2.23617, 1.69454, -0.0735611], [2.3343, 1.75718, -0.120414], [2.42901, 1.82326, -0.169462], [2.5202, 1.89286, -0.22031], [2.60774, 1.96604, -0.27256], [2.69155, 2.04285, -0.325808], [2.77158, 2.12333, -0.37965], [2.84779, 2.20749, -0.433681], [2.92019, 2.29533, -0.487506], [2.98878, 2.38684, -0.540734], [3.05362, 2.48199, -0.592988], [3.11477, 2.58072, -0.643905], [3.17232, 2.68299, -0.693138], [3.22641, 2.78872, -0.740362], [3.27715, 2.89782, -0.785274], [3.32473, 3.0102, -0.827598], [-0.641527, 0.790305, -0.448102], [-0.549457, 0.828733, -0.365703], [-0.453504, 0.86525, -0.287835], [-0.353861, 0.900051, -0.214743], [-0.250748, 0.933342, -0.14665], [-0.144404, 0.965339, -0.0837548], [-0.0350921, 0.996264, -0.0262344], [0.0769107, 1.02635, 0.0257603], [0.19131, 1.05584, 0.0721051], [0.307798, 1.08497, 0.112704], [0.426055, 1.11399, 0.147488], [0.545755, 1.14314, 0.17642], [0.666564, 1.17269, 0.199491], [0.788143, 1.20287, 0.216724], [0.910152, 1.23394, 0.228172], [1.03225, 1.26614, 0.233918], [1.15411, 1.29971, 0.234077], [1.27539, 1.33489, 0.228793], [1.39576, 1.37189, 0.21824], [1.51492, 1.41095, 0.202619], [1.63256, 1.45227, 0.18216], [1.74838, 1.49604, 0.157118], [1.86212, 1.54246, 0.127774], [1.9735, 1.59169, 0.0944322], [2.08228, 1.6439, 0.0574168], [2.18825, 1.69924, 0.0170725], [2.2912, 1.75783, -0.0262393], [2.39094, 1.81979, -0.0721417], [2.48732, 1.88521, -0.120246], [2.5802, 1.95419, -0.170153], [2.66948, 2.02678, -0.221456], [2.75506, 2.10304, -0.273746], [2.83689, 2.18299, -0.326611], [2.91493, 2.26666, -0.379641], [2.98917, 2.35404, -0.432431], [3.05963, 2.44511, -0.484587], [3.12636, 2.53985, -0.535722], [3.18941, 2.6382, -0.585468], [3.24887, 2.74011, -0.633473], [3.30487, 2.84551, -0.679407], [3.35755, 2.95431, -0.722964], [3.40705, 3.0664, -0.763864], [-0.600366, 0.880215, -0.432895], [-0.509247, 0.917373, -0.348831], [-0.414096, 0.952621, -0.269313], [-0.315109, 0.986157, -0.194586], [-0.212507, 1.01819, -0.124874], [-0.106531, 1.04893, -0.060382], [0.00255266, 1.0786, -0.00128694], [0.114464, 1.10745, 0.0522569], [0.228903, 1.1357, 0.100123], [0.345559, 1.16361, 0.142211], [0.464109, 1.19142, 0.178451], [0.58422, 1.21939, 0.208803], [0.705554, 1.24776, 0.233256], [0.827767, 1.27679, 0.251831], [0.950516, 1.30672, 0.26458], [1.07345, 1.33781, 0.271585], [1.19624, 1.37029, 0.272959], [1.31854, 1.4044, 0.268847], [1.44003, 1.44036, 0.259423], [1.56037, 1.47841, 0.244889], [1.67927, 1.51874, 0.225477], [1.79642, 1.56157, 0.201444], [1.91155, 1.60706, 0.173073], [2.02439, 1.65541, 0.14067], [2.13469, 1.70677, 0.104565], [2.24222, 1.76129, 0.0651062], [2.34678, 1.81909, 0.02266], [2.44818, 1.8803, -0.0223915], [2.54626, 1.94502, -0.0696531], [2.64088, 2.01331, -0.118719], [2.73192, 2.08526, -0.169177], [2.8193, 2.16091, -0.220608], [2.90295, 2.24028, -0.272594], [2.98283, 2.3234, -0.324716], [3.05894, 2.41026, -0.376561], [3.13128, 2.50085, -0.427727], [3.19989, 2.59513, -0.477819], [3.26484, 2.69306, -0.526462], [3.32621, 2.79457, -0.573296], [3.38413, 2.89959, -0.617988], [3.43871, 3.00803, -0.660225], [3.49013, 3.1198, -0.699725], [-0.5599, 0.969809, -0.414636], [-0.469731, 1.00564, -0.328939], [-0.375374, 1.03956, -0.247802], [-0.277025, 1.07177, -0.17147], [-0.174909, 1.10248, -0.100173], [-0.0692703, 1.1319, -0.0341136], [0.0396231, 1.16026, 0.0265245], [0.151485, 1.1878, 0.0815846], [0.266013, 1.21475, 0.130936], [0.38289, 1.24137, 0.174478], [0.501787, 1.26791, 0.212135], [0.622368, 1.29461, 0.243864], [0.744287, 1.32173, 0.269653], [0.867198, 1.34953, 0.289521], [0.990748, 1.37826, 0.303516], [1.11459, 1.40815, 0.31172], [1.23837, 1.43947, 0.314247], [1.36176, 1.47244, 0.31124], [1.48441, 1.50729, 0.302872], [1.60601, 1.54425, 0.289349], [1.72623, 1.58353, 0.270902], [1.84477, 1.62533, 0.24779], [1.96135, 1.66984, 0.220301], [2.07571, 1.71723, 0.188743], [2.18757, 1.76767, 0.153449], [2.29672, 1.82131, 0.114773], [2.40294, 1.87826, 0.073087], [2.50604, 1.93866, 0.0287786], [2.60585, 2.0026, -0.0177502], [2.70224, 2.07016, -0.0660864], [2.79508, 2.14141, -0.115809], [2.88428, 2.21639, -0.166491], [2.96977, 2.29513, -0.217705], [3.05151, 2.37766, -0.269023], [3.12949, 2.46396, -0.320023], [3.20372, 2.55401, -0.370291], [3.27423, 2.64779, -0.419426], [3.34108, 2.74524, -0.467041], [3.40436, 2.8463, -0.512772], [3.46418, 2.95089, -0.556274], [3.52067, 3.05894, -0.597231], [3.57398, 3.17033, -0.635359], [-0.520233, 1.05904, -0.393316], [-0.431011, 1.09349, -0.306021], [-0.337435, 1.12602, -0.223296], [-0.239705, 1.15685, -0.145391], [-0.138047, 1.18617, -0.0725366], [-0.0327093, 1.21421, -0.00494146], [0.0760356, 1.24119, 0.0572087], [0.187897, 1.26735, 0.113753], [0.302566, 1.29294, 0.164557], [0.419721, 1.3182, 0.209516], [0.539027, 1.34339, 0.24855], [0.660141, 1.36875, 0.281615], [0.782713, 1.39455, 0.308694], [0.906387, 1.42105, 0.329802], [1.03081, 1.44849, 0.344989], [1.15562, 1.47712, 0.354334], [1.28047, 1.5072, 0.357947], [1.40501, 1.53895, 0.355974], [1.5289, 1.57262, 0.348588], [1.65181, 1.60842, 0.335994], [1.77342, 1.64657, 0.318426], [1.89342, 1.68728, 0.296145], [2.01151, 1.73073, 0.26944], [2.12744, 1.77709, 0.238625], [2.24092, 1.82655, 0.204036], [2.35174, 1.87923, 0.166033], [2.45966, 1.93528, 0.124993], [2.56451, 1.9948, 0.0813109], [2.6661, 2.05791, 0.0353956], [2.76429, 2.12467, -0.0123314], [2.85896, 2.19516, -0.0614401], [2.95, 2.26943, -0.111494], [3.03736, 2.34749, -0.162056], [3.12098, 2.42937, -0.212686], [3.20085, 2.51506, -0.262951], [3.27697, 2.60453, -0.312427], [3.34938, 2.69776, -0.3607], [3.41813, 2.79469, -0.407374], [3.48331, 2.89526, -0.452073], [3.54503, 2.99938, -0.494447], [3.60341, 3.10698, -0.534171], [3.65861, 3.21794, -0.570956], [-0.481478, 1.14785, -0.368931], [-0.393197, 1.18087, -0.280071], [-0.300389, 1.21196, -0.195791], [-0.203254, 1.24134, -0.116343], [-0.102021, 1.26922, -0.0419612], [0.00305662, 1.29581, 0.0271411], [0.111701, 1.32134, 0.090774], [0.223615, 1.34606, 0.148772], [0.338485, 1.37021, 0.200997], [0.455982, 1.39404, 0.247337], [0.575764, 1.41781, 0.287711], [0.697481, 1.44177, 0.322069], [0.820776, 1.46617, 0.350391], [0.945289, 1.49129, 0.372691], [1.07066, 1.51737, 0.389013], [1.19651, 1.54466, 0.399436], [1.32251, 1.57342, 0.40407], [1.44827, 1.60388, 0.403058], [1.57348, 1.63628, 0.396575], [1.69777, 1.67085, 0.384825], [1.82083, 1.7078, 0.368045], [1.94235, 1.74734, 0.346498], [2.06203, 1.78966, 0.320475], [2.17958, 1.83493, 0.290295], [2.29475, 1.88332, 0.256298], [2.40728, 1.93499, 0.218849], [2.51697, 1.99006, 0.178333], [2.6236, 2.04865, 0.135149], [2.72701, 2.11087, 0.0897178], [2.82704, 2.17678, 0.0424677], [2.92357, 2.24646, -0.00616067], [3.01649, 2.31996, -0.0557204], [3.10574, 2.39729, -0.105761], [3.19126, 2.47847, -0.155832], [3.27303, 2.56351, -0.205487], [3.35105, 2.65236, -0.254288], [3.42536, 2.74499, -0.301808], [3.49602, 2.84136, -0.347638], [3.5631, 2.94139, -0.391391], [3.6267, 3.045, -0.432705], [3.68696, 3.15211, -0.471248], [3.74403, 3.26261, -0.506724], [-0.443756, 1.23621, -0.341481], [-0.356411, 1.26773, -0.25109], [-0.264353, 1.29733, -0.165286], [-0.167784, 1.32521, -0.0843242], [-0.0669383, 1.35158, -0.00844335], [0.0379253, 1.37666, 0.0621391], [0.146523, 1.40068, 0.127228], [0.258551, 1.42389, 0.186652], [0.373688, 1.44653, 0.240266], [0.491596, 1.46886, 0.287955], [0.611928, 1.49114, 0.329633], [0.734325, 1.51361, 0.365243], [0.858423, 1.53655, 0.394764], [0.983853, 1.5602, 0.418204], [1.11025, 1.58484, 0.435605], [1.23723, 1.61072, 0.447044], [1.36444, 1.63808, 0.45263], [1.49152, 1.66717, 0.452504], [1.61811, 1.69823, 0.446842], [1.74387, 1.73149, 0.435848], [1.86846, 1.76716, 0.41976], [1.99157, 1.80546, 0.398845], [2.11289, 1.84657, 0.373396], [2.23214, 1.89068, 0.343735], [2.34905, 1.93795, 0.310209], [2.46336, 1.98853, 0.273187], [2.57486, 2.04256, 0.233061], [2.68333, 2.10016, 0.190241], [2.78861, 2.16142, 0.145151], [2.89052, 2.22642, 0.0982337], [2.98894, 2.29524, 0.0499391], [3.08377, 2.36791, 0.000726704], [3.17493, 2.44447, -0.0489399], [3.26236, 2.52492, -0.0985961], [3.34605, 2.60925, -0.14778], [3.42599, 2.69743, -0.196039], [3.50221, 2.78944, -0.24293], [3.57476, 2.8852, -0.288028], [3.64373, 2.98465, -0.330931], [3.70922, 3.08771, -0.371264], [3.77134, 3.19429, -0.408685], [3.83026, 3.30427, -0.44289], [-0.407202, 1.32407, -0.310969], [-0.320786, 1.35405, -0.219082], [-0.229456, 1.3821, -0.131785], [-0.13342, 1.40841, -0.0493377], [-0.0329169, 1.43321, 0.0280174], [0.0717867, 1.45672, 0.100056], [0.1804, 1.47916, 0.166577], [0.29261, 1.5008, 0.227401], [0.408088, 1.52186, 0.282379], [0.526486, 1.54262, 0.331387], [0.647449, 1.56332, 0.374333], [0.770611, 1.58424, 0.411158], [0.895597, 1.60562, 0.441832], [1.02203, 1.62775, 0.466362], [1.14954, 1.65087, 0.484786], [1.27774, 1.67524, 0.497179], [1.40625, 1.70113, 0.503646], [1.53472, 1.72877, 0.504329], [1.66279, 1.75841, 0.499403], [1.79009, 1.79028, 0.489073], [1.91629, 1.8246, 0.473578], [2.04107, 1.86157, 0.453186], [2.16411, 1.90141, 0.428196], [2.28513, 1.94427, 0.398933], [2.40384, 1.99035, 0.365748], [2.51999, 2.03978, 0.329018], [2.63336, 2.09271, 0.28914], [2.74372, 2.14924, 0.246533], [2.85091, 2.20949, 0.201633], [2.95474, 2.27354, 0.15489], [3.0551, 2.34144, 0.106769], [3.15186, 2.41324, 0.0577412], [3.24496, 2.48897, 0.00828567], [3.33432, 2.56864, -0.0411169], [3.41994, 2.65223, -0.0899877], [3.5018, 2.73971, -0.137854], [3.57993, 2.83104, -0.184256], [3.65438, 2.92617, -0.22875], [3.72524, 3.02501, -0.270914], [3.7926, 3.12748, -0.310358], [3.85658, 3.23348, -0.346726], [3.91733, 3.34291, -0.379703], [-0.371965, 1.41139, -0.277407], [-0.286465, 1.43978, -0.184059], [-0.195837, 1.46622, -0.0952987], [-0.100295, 1.49091, -0.0113897], [-8.30281e-05, 1.51408, 0.0674182], [0.104523, 1.53595, 0.140894], [0.213222, 1.55676, 0.208826], [0.325692, 1.57674, 0.271031], [0.441593, 1.59616, 0.327349], [0.560569, 1.61527, 0.377649], [0.682254, 1.63433, 0.421834], [0.806271, 1.65361, 0.459836], [0.93224, 1.67337, 0.491622], [1.05977, 1.69388, 0.517193], [1.18849, 1.7154, 0.536584], [1.31799, 1.73819, 0.549866], [1.44791, 1.76252, 0.557144], [1.57787, 1.78863, 0.558557], [1.70749, 1.81677, 0.554279], [1.83642, 1.84717, 0.544517], [1.96432, 1.88005, 0.529511], [2.09084, 1.91563, 0.509531], [2.21569, 1.9541, 0.484878], [2.33854, 1.99566, 0.455883], [2.45913, 2.04046, 0.422902], [2.57719, 2.08867, 0.386317], [2.69248, 2.14043, 0.346535], [2.80479, 2.19585, 0.303982], [2.91394, 2.25503, 0.259104], [3.01974, 2.31805, 0.212365], [3.12206, 2.38499, 0.16424], [3.2208, 2.45587, 0.115217], [3.31585, 2.53073, 0.0657917], [3.40717, 2.60958, 0.0164623], [3.49473, 2.69239, -0.0322722], [3.57852, 2.77913, -0.0799183], [3.65857, 2.86976, -0.125993], [3.73492, 2.96421, -0.170029], [3.80765, 3.06241, -0.211584], [3.87687, 3.16425, -0.250245], [3.94268, 3.26965, -0.285639], [4.00525, 3.37849, -0.317439], [-0.338209, 1.49815, -0.240814], [-0.25361, 1.52489, -0.14604], [-0.163653, 1.54966, -0.0558421], [-0.0685554, 1.57268, 0.029508], [0.0314262, 1.59416, 0.109753], [0.136006, 1.61433, 0.184651], [0.244873, 1.63343, 0.253982], [0.357691, 1.6517, 0.317552], [0.474109, 1.6694, 0.375192], [0.593759, 1.68679, 0.426763], [0.716265, 1.70413, 0.472159], [0.84124, 1.72169, 0.511306], [0.968293, 1.73974, 0.544164], [1.09703, 1.75855, 0.570729], [1.22705, 1.77839, 0.591032], [1.35797, 1.79952, 0.60514], [1.48938, 1.82221, 0.613156], [1.62092, 1.8467, 0.615218], [1.7522, 1.87325, 0.611499], [1.88286, 1.90209, 0.602206], [2.01254, 1.93346, 0.587579], [2.1409, 1.96756, 0.567892], [2.26762, 2.0046, 0.54345], [2.3924, 2.04476, 0.514586], [2.51494, 2.08823, 0.481662], [2.63497, 2.13515, 0.445068], [2.75226, 2.18566, 0.405217], [2.86658, 2.2399, 0.362546], [2.97773, 2.29795, 0.317511], [3.08554, 2.3599, 0.270587], [3.18988, 2.42582, 0.222265], [3.29061, 2.49574, 0.17305], [3.38766, 2.56969, 0.123453], [3.48095, 2.64768, 0.0739946], [3.57047, 2.72967, 0.0251958], [3.65619, 2.81565, -0.0224251], [3.73815, 2.90555, -0.0683582], [3.8164, 2.9993, -0.112109], [3.891, 3.09682, -0.153206], [3.96206, 3.19801, -0.19121], [4.0297, 3.30277, -0.225725], [4.09406, 3.41099, -0.256405], [-0.306115, 1.58431, -0.201221], [-0.222399, 1.60935, -0.105052], [-0.133072, 1.63241, -0.0134393], [-0.038362, 1.65369, 0.0733377], [0.0614626, 1.67342, 0.15501], [0.166102, 1.69183, 0.231325], [0.275229, 1.70915, 0.302049], [0.388495, 1.72564, 0.366977], [0.505535, 1.74154, 0.425928], [0.625968, 1.75713, 0.478754], [0.749405, 1.77267, 0.525339], [0.875449, 1.78844, 0.565601], [1.0037, 1.8047, 0.599494], [1.13375, 1.82173, 0.627008], [1.26519, 1.8398, 0.648169], [1.39762, 1.85918, 0.663041], [1.53065, 1.88014, 0.671723], [1.66387, 1.90293, 0.674351], [1.79691, 1.9278, 0.671098], [1.9294, 1.95501, 0.662171], [2.06096, 1.98477, 0.647812], [2.19125, 2.0173, 0.628295], [2.31994, 2.05283, 0.603929], [2.44672, 2.09152, 0.575051], [2.57129, 2.13357, 0.54203], [2.69337, 2.17913, 0.505261], [2.81272, 2.22834, 0.465167], [2.9291, 2.28133, 0.422191], [3.04232, 2.33819, 0.376804], [3.1522, 2.39902, 0.329493], [3.25858, 2.46387, 0.280764], [3.36136, 2.53279, 0.231137], [3.46042, 2.60579, 0.181145], [3.55571, 2.68288, 0.131331], [3.64719, 2.76403, 0.0822409], [3.73486, 2.84921, 0.0344215], [3.81873, 2.93836, -0.0115859], [3.89886, 3.03139, -0.0552526], [3.97532, 3.12822, -0.0960719], [4.04821, 3.22874, -0.13357], [4.11765, 3.33283, -0.167318], [4.18378, 3.44038, -0.196947], [-0.275889, 1.66984, -0.15867], [-0.193031, 1.69313, -0.0611361], [-0.104283, 1.71442, 0.0318758], [-0.00988943, 1.73392, 0.120074], [0.0898657, 1.75184, 0.203175], [0.194664, 1.76842, 0.280911], [0.30416, 1.7839, 0.353033], [0.417989, 1.79853, 0.419319], [0.535768, 1.81257, 0.479579], [0.657105, 1.82629, 0.53365], [0.781595, 1.83995, 0.581408], [0.908829, 1.85384, 0.622761], [1.03839, 1.86822, 0.657657], [1.16988, 1.88338, 0.686077], [1.30286, 1.89959, 0.708045], [1.43693, 1.91713, 0.723618], [1.57168, 1.93627, 0.732893], [1.7067, 1.95727, 0.736004], [1.84161, 1.98038, 0.733123], [1.97603, 2.00585, 0.724456], [2.10957, 2.03392, 0.710247], [2.2419, 2.06481, 0.690771], [2.37265, 2.09873, 0.666341], [2.50152, 2.13588, 0.637298], [2.6282, 2.17643, 0.604016], [2.75242, 2.22056, 0.566898], [2.8739, 2.26839, 0.526372], [2.99242, 2.32007, 0.482895], [3.10776, 2.37568, 0.436946], [3.21975, 2.43533, 0.389027], [3.32823, 2.49906, 0.339659], [3.43308, 2.56693, 0.289381], [3.53418, 2.63895, 0.238747], [3.63149, 2.71512, 0.188323], [3.72495, 2.79541, 0.138684], [3.81456, 2.87977, 0.0904093], [3.90035, 2.96815, 0.0440764], [3.98235, 3.06045, 0.000255219], [4.06065, 3.15657, -0.0405028], [4.13535, 3.2564, -0.0776778], [4.20657, 3.3598, -0.110798], [4.27446, 3.46666, -0.139459], [-0.247764, 1.75472, -0.113222], [-0.165729, 1.77622, -0.0143482], [-0.0774963, 1.79569, 0.0800569], [0.0166705, 1.81334, 0.169684], [0.116463, 1.8294, 0.254228], [0.221539, 1.84409, 0.333402], [0.33153, 1.85766, 0.406937], [0.446052, 1.87037, 0.474596], [0.564704, 1.88247, 0.53617], [0.687077, 1.89423, 0.591486], [0.812753, 1.90593, 0.640407], [0.941311, 1.91785, 0.682833], [1.07233, 1.93027, 0.718703], [1.20537, 1.94347, 0.747991], [1.34002, 1.95773, 0.770715], [1.47585, 1.97334, 0.786929], [1.61245, 1.99056, 0.796724], [1.7494, 2.00967, 0.800235], [1.8863, 2.03092, 0.797629], [2.02276, 2.05457, 0.789114], [2.1584, 2.08086, 0.774933], [2.29285, 2.11001, 0.755366], [2.42578, 2.14225, 0.730725], [2.55684, 2.17776, 0.701358], [2.68573, 2.21674, 0.667642], [2.81215, 2.25935, 0.629988], [2.93584, 2.30574, 0.588833], [3.05657, 2.35604, 0.544642], [3.1741, 2.41035, 0.497907], [3.28826, 2.46876, 0.449142], [3.39889, 2.53133, 0.398885], [3.50584, 2.59811, 0.347693], [3.60902, 2.66911, 0.296143], [3.70836, 2.74433, 0.244826], [3.80381, 2.82374, 0.194347], [3.89537, 2.90728, 0.145321], [3.98305, 2.99488, 0.0983682], [4.06692, 3.08645, 0.054108], [4.14704, 3.18186, 0.0131473], [4.22352, 3.28099, -0.023933], [4.2965, 3.3837, -0.0566], [4.36611, 3.48985, -0.0843967], [-0.222008, 1.83894, -0.0649635], [-0.140749, 1.85859, 0.0352351], [-0.0529456, 1.87619, 0.131043], [0.0411083, 1.89195, 0.222124], [0.14107, 1.90609, 0.308145], [0.246565, 1.91883, 0.388792], [0.357198, 1.93042, 0.463772], [0.472561, 1.94113, 0.532828], [0.592234, 1.95121, 0.595734], [0.715792, 1.96094, 0.652304], [0.842801, 1.9706, 0.702387], [0.972828, 1.98046, 0.745874], [1.10543, 1.99083, 0.782693], [1.24018, 2.00197, 0.812815], [1.37664, 2.01419, 0.836248], [1.51437, 2.02777, 0.853042], [1.65294, 2.04298, 0.863287], [1.79194, 2.0601, 0.867111], [1.93096, 2.07939, 0.864683], [2.06959, 2.10112, 0.856209], [2.20744, 2.12553, 0.841932], [2.34415, 2.15285, 0.822134], [2.47935, 2.18331, 0.79713], [2.6127, 2.2171, 0.767272], [2.74389, 2.25442, 0.732942], [2.87261, 2.29544, 0.694557], [2.9986, 2.34031, 0.652562], [3.12161, 2.38916, 0.607432], [3.2414, 2.4421, 0.55967], [3.35779, 2.49922, 0.509804], [3.4706, 2.56059, 0.458387], [3.57971, 2.62624, 0.405996], [3.68499, 2.69619, 0.353227], [3.78638, 2.77044, 0.300701], [3.88384, 2.84895, 0.249052], [3.97734, 2.93167, 0.198936], [4.06692, 3.0185, 0.15102], [4.15262, 3.10935, 0.105979], [4.23453, 3.20407, 0.0644893], [4.31276, 3.30252, 0.0272131], [4.38746, 3.40453, -0.00522921], [4.45877, 3.50995, -0.0322987], [-0.198937, 1.92248, -0.0140127], [-0.118387, 1.94024, 0.0875102], [-0.0308932, 1.95593, 0.184755], [0.0631968, 1.96974, 0.277342], [0.163491, 1.98189, 0.364899], [0.269575, 1.99263, 0.447074], [0.381021, 2.00218, 0.523549], [0.497394, 2.01082, 0.594043], [0.618254, 2.0188, 0.658312], [0.743159, 2.02641, 0.716154], [0.87166, 2.03394, 0.767406], [1.00331, 2.04166, 0.811948], [1.13766, 2.04987, 0.849699], [1.27427, 2.05887, 0.880623], [1.41268, 2.06894, 0.904721], [1.55245, 2.08038, 0.922039], [1.69314, 2.09348, 0.932662], [1.83434, 2.10851, 0.936715], [1.9756, 2.12574, 0.934366], [2.11653, 2.14545, 0.925818], [2.25672, 2.16788, 0.911318], [2.3958, 2.19327, 0.891146], [2.53339, 2.22185, 0.865621], [2.66914, 2.25383, 0.835098], [2.80274, 2.28941, 0.799965], [2.93386, 2.32875, 0.760643], [3.06224, 2.37202, 0.717587], [3.1876, 2.41935, 0.67128], [3.30973, 2.47086, 0.622235], [3.4284, 2.52664, 0.570995], [3.54347, 2.58675, 0.518126], [3.65476, 2.65123, 0.464225], [3.76218, 2.72011, 0.409909], [3.86565, 2.79338, 0.355823], [3.96511, 2.87099, 0.302637], [4.06055, 2.95289, 0.251043], [4.152, 3.03897, 0.201759], [4.23952, 3.12912, 0.155524], [4.32318, 3.22318, 0.113098], [4.40312, 3.32099, 0.0752471], [4.47949, 3.42233, 0.0427176], [4.55246, 3.52702, 0.0161839], [-0.178938, 2.00532, 0.0394602], [-0.0989873, 2.02116, 0.142337], [-0.0116315, 2.03489, 0.241093], [0.0826935, 2.0467, 0.335278], [0.183526, 2.05683, 0.424461], [0.290401, 2.06549, 0.508248], [0.402856, 2.07293, 0.586289], [0.520429, 2.07942, 0.658278], [0.64266, 2.08523, 0.723953], [0.769089, 2.09064, 0.783097], [0.899256, 2.09594, 0.835534], [1.0327, 2.10142, 0.881132], [1.16896, 2.10738, 0.919804], [1.30758, 2.11413, 0.951503], [1.4481, 2.12195, 0.976227], [1.59007, 2.13115, 0.994013], [1.73304, 2.14203, 1.00494], [1.87657, 2.15486, 1.00914], [2.02022, 2.16992, 1.00677], [2.16359, 2.1875, 0.998034], [2.30626, 2.20784, 0.983179], [2.44783, 2.2312, 0.962486], [2.58793, 2.25781, 0.936276], [2.72621, 2.28788, 0.904908], [2.86232, 2.32162, 0.868774], [2.99595, 2.3592, 0.828301], [3.12681, 2.40079, 0.783951], [3.25462, 2.44653, 0.736216], [3.37915, 2.49654, 0.685619], [3.50019, 2.55091, 0.632713], [3.61756, 2.60972, 0.578082], [3.73109, 2.673, 0.522337], [3.84068, 2.74079, 0.466116], [3.94624, 2.81305, 0.410089], [4.04772, 2.88977, 0.354954], [4.1451, 2.97086, 0.301443], [4.2384, 3.05622, 0.250322], [4.32768, 3.14573, 0.202399], [4.41305, 3.2392, 0.158522], [4.49463, 3.33642, 0.119588], [4.57261, 3.43716, 0.0865239], [4.64719, 3.54115, 0.0602321], [-0.162505, 2.08745, 0.0952026], [-0.0829602, 2.10133, 0.199529], [0.00451981, 2.11307, 0.299942], [0.0993493, 2.12285, 0.395871], [0.200977, 2.13089, 0.486812], [0.308882, 2.13742, 0.572323], [0.422569, 2.14269, 0.652023], [0.541552, 2.14696, 0.725582], [0.665353, 2.15051, 0.792721], [0.793499, 2.15363, 0.853207], [0.925516, 2.15661, 0.906853], [1.06093, 2.15975, 0.953518], [1.19928, 2.16336, 0.993104], [1.34008, 2.16774, 1.02556], [1.48288, 2.17321, 1.05087], [1.62721, 2.18006, 1.06907], [1.77261, 2.18859, 1.08024], [1.91863, 2.19911, 1.0845], [2.06484, 2.2119, 1.082], [2.21079, 2.22723, 1.07296], [2.35607, 2.24537, 1.05762], [2.50028, 2.26659, 1.03625], [2.64304, 2.29111, 1.00919], [2.78396, 2.31917, 0.976791], [2.92271, 2.35097, 0.939451], [3.05895, 2.3867, 0.897604], [3.19239, 2.42653, 0.851717], [3.32275, 2.4706, 0.80229], [3.44977, 2.51904, 0.749856], [3.57325, 2.57195, 0.69498], [3.69298, 2.62941, 0.638257], [3.8088, 2.69145, 0.580312], [3.9206, 2.75811, 0.521803], [4.02828, 2.82937, 0.46342], [4.13178, 2.90519, 0.405887], [4.23108, 2.9855, 0.349969], [4.3262, 3.0702, 0.296475], [4.41721, 3.15913, 0.246273], [4.50421, 3.2521, 0.200301], [4.58735, 3.34886, 0.159597], [4.66683, 3.44912, 0.12532], [4.74293, 3.5525, 0.0987585], [-0.150327, 2.16885, 0.152814], [-0.0707834, 2.18077, 0.258846], [0.0172338, 2.19049, 0.361179], [0.112928, 2.19821, 0.459073], [0.215661, 2.20412, 0.551949], [0.324871, 2.20846, 0.639327], [0.440038, 2.21148, 0.7208], [0.560659, 2.21345, 0.79602], [0.686245, 2.21465, 0.864693], [0.816311, 2.21539, 0.926572], [0.950374, 2.21595, 0.981462], [1.08795, 2.21665, 1.02921], [1.22857, 2.21779, 1.06971], [1.37174, 2.2197, 1.1029], [1.517, 2.22269, 1.12876], [1.66386, 2.22707, 1.14733], [1.81186, 2.23315, 1.15868], [1.96054, 2.24123, 1.16291], [2.10945, 2.25161, 1.16019], [2.25815, 2.26458, 1.15073], [2.4062, 2.28041, 1.13476], [2.5532, 2.29937, 1.11257], [2.69874, 2.3217, 1.08448], [2.84245, 2.34764, 1.05085], [2.98396, 2.3774, 1.0121], [3.12294, 2.41117, 0.968643], [3.25907, 2.44914, 0.920966], [3.39207, 2.49146, 0.869574], [3.52168, 2.53826, 0.815007], [3.64767, 2.58965, 0.75784], [3.76984, 2.64569, 0.698678], [3.88801, 2.70646, 0.638159], [4.00205, 2.77197, 0.576956], [4.11187, 2.84221, 0.515775], [4.2174, 2.91715, 0.455361], [4.31861, 2.99672, 0.3965], [4.41553, 3.0808, 0.340034], [4.5082, 3.16925, 0.286873], [4.59674, 3.26185, 0.238021], [4.68131, 3.35833, 0.194625], [4.76215, 3.45834, 0.158071], [4.8396, 3.56135, 0.13019]],dtype=np.float)

# change to viewer coordinate system: from [x,y,z] with z up -> [x,z,-y] with y up
V = V[:,[0,2,1]]
V[:,2] *= -1

Q = get_grid_quads(nrows, ncols)

F = quads2tris(Q)

p = mp.plot(V, F, return_plot=True)
# p = mp.plot(V, F, shading={"wireframe": True}, return_plot=True)
p.add_edges(V,quads2edges(Q))

p.add_points(np.array([[0,0,0]]), shading={"point_color": "black"})
add_axes(p, l=2)

In [43]:
import numpy as np
import hFEMs as hs
import meshplot as mp

vegfile = "/home/gsperl/Projects/hFEMs/meshes/testmesh.veg"
# strain = np.array([0.5,0,0, 3,0,0],dtype=np.float)
strain = np.array([0,0,0, 5,0,0],dtype=np.float)

solvable = hs.VegaFEMSolvable(vegfile, strain)

nssettings = hs.NewtonSolverSettings()
nssettings.projgrad_epsilon = 1e-2
nssettings.reg_start = 1e5
nssettings.reg_end = 1e0
nssettings.reg_steps = 5000
nssettings.step_limit = 1e-3
ns = hs.NewtonSolver(solvable, nssettings)

N = 0
# N = 10000
i = 0
for i in range(N):
    if ns.isFinished():
        break
    ns.step()
print("Finished after", i)

        
x = solvable.getWorldPositions(ns.getSolution())
# nrows,ncols,phi = solvable.getGrid() # THIS MIGHT BE A MEMORY LEAK BC POINTERS


Finished after 0


In [44]:
F = tets2tris(T)
Vdef = x.reshape(-1,3)
p = mp.plot(Vdef @ Raxes.T, F, shading={"wireframe": True}, return_plot=True)
add_axes(p, l=1, R=Raxes)
# add_grid_plot(p, nrows, ncols, phi)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.04…

In [1854]:
import numpy as np
import hFEMs as hs
import meshplot as mp

vegfile = "/home/gsperl/Projects/hFEMs/meshes/testmesh.veg"
# strain = np.array([0.5,0,0, 3,0,0],dtype=np.float)
# strain = np.array([0,0,0, 5,0,0],dtype=np.float)
# strain = np.array([-0.6,0.0,0, 0,0,0],dtype=np.float)
strain = np.array([0,0.0,0, 0,0,4],dtype=np.float)

solvable = hs.VegaFEMSolvable(vegfile, strain)

nssettings = hs.NewtonSolverSettings()
nssettings.projgrad_epsilon = 1e-10
nssettings.reg_start = 1e5
nssettings.reg_end = 1e0
nssettings.reg_steps = 5000
nssettings.step_limit = 1e-3
ns = hs.NewtonSolver(solvable, nssettings)

F = tets2tris(T) # ASSUME T LOADED FROM SOMEWHERE # could also make F = a get_boundary_set triangles method
V = solvable.getWorldPositions(ns.getSolution()).reshape(-1,3)

# p = mp.plot(V @ Raxes.T, F, shading={"wireframe": True}, return_plot=True)
p = mp.plot(V @ Raxes.T, F, shading={"wireframe": False}, return_plot=True)
# p = mp.plot(V @ Raxes.T, c=np.random.rand(*V.shape),shading={"point_size": 0.1}, return_plot=True)
add_axes(p, l=0.5, R=Raxes)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.02…

In [1895]:
for i in range(100):
    if ns.isFinished():
        print("FINISHED")
        break
    else:
        ns.step()
V = solvable.getWorldPositions(ns.getSolution()).reshape(-1,3)
p.update_object(oid=0, vertices=V @ Raxes.T)


FINISHED
